## Common Section

In [1]:
# Google Drive Operations - Only for Google Drive, Delete in Local Settings
# Reference for using R in Colab: https://towardsdatascience.com/how-to-use-r-in-google-colab-b6e02d736497

%load_ext rpy2.ipython
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/Versed Chimpanzee - AC Group")
!ls


Mounted at /content/drive
'AC Use Cases Draft.pdf'	     interim_data
 customers.csv			     Plan.gdoc
'data columns with colors.pdf'	     submission_random.csv
'Feature Importance Scores.gsheet'   transactions.csv
 geo.csv			    'Versed Chimpanzee.ipynb'
'Info - Analytics Cup 2022.pdf'


## 1. Data Preprocessing

### 1.1 Prepare Libraries

In [2]:
%%R

library(dplyr, warn.conflicts = F, quietly = T)
library(tidyverse, warn.conflicts = F, quietly = T)
library(lubridate, warn.conflicts = F, quietly = T)

R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

R[write to console]: ✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ stringr 1.4.0
✔ tidyr   1.1.4     ✔ forcats 0.5.1
✔ readr   2.1.1     

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



### 1.2 Load and Inspect Transactions (transactions.csv)

In [3]:
%%R

trnsc_df = as_tibble(read_csv("transactions.csv"))

print("[INFO] Number of NA values in each column:")
for (i in 1:ncol(trnsc_df)) {
  print(paste0(names(trnsc_df)[i], ": ", sum(is.na(trnsc_df[, i])), "/", nrow(trnsc_df)))
}

trnsc_df$CUSTOMER = (substring(trnsc_df$CUSTOMER, 2, nchar(trnsc_df$CUSTOMER)-1))

Rows: 26151 Columns: 23
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (12): MO_ID, SO_ID, CUSTOMER, END_CUSTOMER, PRICE_LIST, MO_CREATED_DATE,...
dbl (11): OFFER_PRICE, SERVICE_LIST_PRICE, MATERIAL_COST, SERVICE_COST, ISIC...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
[1] "[INFO] Number of NA values in each column:"
[1] "MO_ID: 0/26151"
[1] "SO_ID: 0/26151"
[1] "CUSTOMER: 0/26151"
[1] "END_CUSTOMER: 20114/26151"
[1] "OFFER_PRICE: 0/26151"
[1] "SERVICE_LIST_PRICE: 0/26151"
[1] "MATERIAL_COST: 0/26151"
[1] "SERVICE_COST: 0/26151"
[1] "PRICE_LIST: 0/26151"
[1] "ISIC: 1675/26151"
[1] "MO_CREATED_DATE: 0/26151"
[1] "SO_CREATED_DATE: 0/26151"
[1] "TECH: 0/26151"
[1] "OFFER_TYPE: 0/26151"
[1] "BUSINESS_TYPE: 0/26151"
[1] "COSTS_PRODUCT_A: 0/26151"
[1] "COSTS_PRODUCT_B: 0/26151"
[1] "COSTS_PRODUCT_C: 0/26151"
[1] "OFFER_STATUS: 25

### 1.3 Load, Inspect and Merge Geographic Data (geo.csv)

In [4]:
%%R

geo_df = as_tibble(read_csv("geo.csv"))

print("[INFO] Number of NA values in each column:")
for (i in 1:ncol(geo_df)) {
  print(paste0(names(geo_df)[i], ": ", sum(is.na(geo_df[, i])), "/", nrow(geo_df)))
}
# Rename column COUNTRY COUNTRY_CODE, since it only contains codes like CH, FR
geo_df = rename(geo_df, COUNTRY_CODE = COUNTRY)

# Perform left join using dplyr
trnsc_geo_df = left_join(trnsc_df, geo_df, by = 'SALES_LOCATION')

# FIXME: Delete this before submission.
write_csv(x = trnsc_geo_df, file = "interim_data/trnsc_geo_df.csv")


Rows: 46 Columns: 4
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): COUNTRY, SALES_OFFICE, SALES_BRANCH, SALES_LOCATION

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
[1] "[INFO] Number of NA values in each column:"
[1] "COUNTRY: 0/46"
[1] "SALES_OFFICE: 2/46"
[1] "SALES_BRANCH: 1/46"
[1] "SALES_LOCATION: 1/46"


### 1.4 Load, Inspect and Merge Customers Data (customers.csv)

In [5]:
%%R

cst_df = as_tibble(read_csv("customers.csv"))

print("[INFO] Number of NA values in each column:")
for (i in 1:ncol(cst_df)) {
  print(paste0(names(cst_df)[i], ": ", sum(is.na(cst_df[, i])), "/", nrow(cst_df)))
}

##### NOTE FROM TEO: I THINK THE MERGE MAY BE WRONG, I'M MARKING THE SECTION ##########
##### WHICH I THINK MAY NEED REPLACEMENT (SEE MY SECTION) #################
##### REPLACE FROM HERE ###################
# Change data type of CUSTOMER column in customers dataset
cst_df$CUSTOMER <- as.character(cst_df$CUSTOMER)

# Create IDX_CUSTOMER for trnsc_geo_df
trnsc_geo_df = mutate(trnsc_geo_df,
                      IDX_CUSTOMER = paste0(COUNTRY_CODE, "_", CUSTOMER))

# Create IDX_CUSTOMER for customers
cst_df = cst_df %>% mutate(COUNTRY_CODE = case_when(
  COUNTRY == 'Switzerland' ~ "CH",
  COUNTRY == 'France' ~ "FR"
))

cst_df = mutate(cst_df, IDX_CUSTOMER = paste0(COUNTRY_CODE, "_", CUSTOMER))

# Perform left join using dplyr
all_merged = left_join(trnsc_geo_df, cst_df, by = 'IDX_CUSTOMER')
####### TO HERE ###############################

# FIXME: Delete this before submisssion.
write_csv(x = all_merged, file = "interim_data/all_merged.csv")

Rows: 8452 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (5): REV_CURRENT_YEAR, CREATION_YEAR, OWNERSHIP, COUNTRY, CURRENCY
dbl (3): CUSTOMER, REV_CURRENT_YEAR.1, REV_CURRENT_YEAR.2

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
[1] "[INFO] Number of NA values in each column:"
[1] "CUSTOMER: 0/8452"
[1] "REV_CURRENT_YEAR: 0/8452"
[1] "REV_CURRENT_YEAR.1: 0/8452"
[1] "REV_CURRENT_YEAR.2: 0/8452"
[1] "CREATION_YEAR: 0/8452"
[1] "OWNERSHIP: 0/8452"
[1] "COUNTRY: 0/8452"
[1] "CURRENCY: 0/8452"


### 1.5 Fix Basic Problems in Merged Data

In [6]:
%%R

print(paste("[INFO] Number Of Cols:", ncol(all_merged)))
print(paste("[INFO] Number Of Rows:", nrow(all_merged)))
cat("[INFO] Names Of Columns:\n", sprintf("\"%s\", ", names(all_merged)))

cat("\n\n----------------------------------------------------\n\n")

print("[INFO] Glimpse:")
glimpse(all_merged)

cat("\n----------------------------------------------------\n\n")

print("[INFO] First 20 Rows:")
print(head(all_merged, n = 20))

[1] "[INFO] Number Of Cols: 36"
[1] "[INFO] Number Of Rows: 26151"
[INFO] Names Of Columns:
 "MO_ID",  "SO_ID",  "CUSTOMER.x",  "END_CUSTOMER",  "OFFER_PRICE",  "SERVICE_LIST_PRICE",  "MATERIAL_COST",  "SERVICE_COST",  "PRICE_LIST",  "ISIC",  "MO_CREATED_DATE",  "SO_CREATED_DATE",  "TECH",  "OFFER_TYPE",  "BUSINESS_TYPE",  "COSTS_PRODUCT_A",  "COSTS_PRODUCT_B",  "COSTS_PRODUCT_C",  "OFFER_STATUS",  "COSTS_PRODUCT_D",  "COSTS_PRODUCT_E",  "SALES_LOCATION",  "TEST_SET_ID",  "COUNTRY_CODE.x",  "SALES_OFFICE",  "SALES_BRANCH",  "IDX_CUSTOMER",  "CUSTOMER.y",  "REV_CURRENT_YEAR",  "REV_CURRENT_YEAR.1",  "REV_CURRENT_YEAR.2",  "CREATION_YEAR",  "OWNERSHIP",  "COUNTRY",  "CURRENCY",  "COUNTRY_CODE.y", 

----------------------------------------------------

[1] "[INFO] Glimpse:"
Rows: 26,151
Columns: 36
$ MO_ID              <chr> "a050N000013fnfrQAA", "a050N000013fgL1QAI", "a050N0…
$ SO_ID              <chr> "a030N00001EochoQAB", "a030N00001EociNQAR", "a030N0…
$ CUSTOMER.x         <chr> "1", "

In [7]:
%%R

# Feature Elimination & Fix
all_merged = select(all_merged, -c(COUNTRY_CODE.y, CUSTOMER.y, COUNTRY, MO_ID, SO_ID))

# Feature Renaming
all_merged = rename(all_merged, COUNTRY_CODE = COUNTRY_CODE.x)
all_merged = rename(all_merged, CUSTOMER = CUSTOMER.x)

In [8]:
%%R

print("[INFO] Number of NA values in each column:")
for (i in 1:ncol(all_merged)) { # for-loop over columns
  print(paste0(names(all_merged)[i], ": ", sum(is.na(all_merged[, i])), "/", nrow(all_merged)))
}

[1] "[INFO] Number of NA values in each column:"
[1] "CUSTOMER: 0/26151"
[1] "END_CUSTOMER: 20114/26151"
[1] "OFFER_PRICE: 0/26151"
[1] "SERVICE_LIST_PRICE: 0/26151"
[1] "MATERIAL_COST: 0/26151"
[1] "SERVICE_COST: 0/26151"
[1] "PRICE_LIST: 0/26151"
[1] "ISIC: 1675/26151"
[1] "MO_CREATED_DATE: 0/26151"
[1] "SO_CREATED_DATE: 0/26151"
[1] "TECH: 0/26151"
[1] "OFFER_TYPE: 0/26151"
[1] "BUSINESS_TYPE: 0/26151"
[1] "COSTS_PRODUCT_A: 0/26151"
[1] "COSTS_PRODUCT_B: 0/26151"
[1] "COSTS_PRODUCT_C: 0/26151"
[1] "OFFER_STATUS: 2576/26151"
[1] "COSTS_PRODUCT_D: 0/26151"
[1] "COSTS_PRODUCT_E: 0/26151"
[1] "SALES_LOCATION: 37/26151"
[1] "TEST_SET_ID: 23575/26151"
[1] "COUNTRY_CODE: 0/26151"
[1] "SALES_OFFICE: 38/26151"
[1] "SALES_BRANCH: 37/26151"
[1] "IDX_CUSTOMER: 0/26151"
[1] "REV_CURRENT_YEAR: 2885/26151"
[1] "REV_CURRENT_YEAR.1: 2885/26151"
[1] "REV_CURRENT_YEAR.2: 2885/26151"
[1] "CREATION_YEAR: 2885/26151"
[1] "OWNERSHIP: 2885/26151"
[1] "CURRENCY: 2885/26151"


In [9]:
%%R

print("[INFO] Unique values in each column:")
for (i in 1:ncol(all_merged)) {
  count_unq_vals = count(unique(all_merged[, i]))
  if (count_unq_vals <20) {
    print(paste0(names(all_merged)[i], ": ", unique(all_merged[, i])))
    print("---------------")
  }
}

# FIXME: Delete this before submission.
write_csv(x = all_merged, file = "interim_data/all_merged_2.csv")

[1] "[INFO] Unique values in each column:"
[1] "PRICE_LIST: c(\"SFT Standard\", \"CMT Installer\", \"CMT End Customer\", \"Tarif public\")"
[1] "---------------"
[1] "TECH: c(\"S\", \"C\", \"F\", \"BP\", \"FP\", \"EPS\", \"E\")"
[1] "---------------"
[1] "BUSINESS_TYPE: c(\"E\", \"N\", \"M\", \"C\", \"T\", \"Exp\", \"New\", \"Mig\", \"S\", \"F\", \"R\")"
[1] "---------------"
[1] "OFFER_STATUS: c(\"LOsT\", \"Lost\", \"WIN\", \"Win\", NA, \"Won\", \"LOST\", \"Lose\", \"WON\")"
[1] "---------------"
[1] "COUNTRY_CODE: c(\"CH\", \"FR\")"
[1] "---------------"
[1] "SALES_BRANCH: c(\"Branch Central\", \"Branch East\", \"Branch West\", \"EPS CH\", NA, \"Grand Paris\", \"Sud Ouest\", \"Nord FR\", \"Ouest\", \"Centre-Est\", \"Grand Est\", \"Sud-Est\", \"Enterprise Business France\", \"SI\")"
[1] "---------------"
[1] "OWNERSHIP: c(\"Privately Owned/Publicly Traded\", NA, \"Governmental\", \"No information\", \"Individual Person\")"
[1] "---------------"
[1] "CURRENCY: c(\"Chinese Yuan\", NA, \

In [10]:
%%R

merged_df = all_merged
write_csv(x = all_merged, file = "interim_data/merged_df.csv")

### 1.6 Split Data into Labeled and Unlabeled (Test) Sets

In [11]:
%%R

## TODO: YOU CAN SPLIT AFTERWARDS
labeled_data = all_merged[is.na(all_merged$TEST_SET_ID),]
test_data = all_merged[!is.na(all_merged$TEST_SET_ID),]

# FIXME: Delete this before submission.
write_csv(x = test_data, file = "interim_data/labeled_set.csv")
write_csv(x = test_data, file = "interim_data/test_set.csv")

labeled_data %>% nrow()
test_data %>% nrow()

[1] 2576


## 2. Feature Elimination and Manipulation

In [12]:
%%R

df = data.frame(merged_df)

### 2.1 Delete Unnecessary Columns

In [13]:
%%R

# Delete columns: MO_CREATED_DATE, SALES_BRANCH, REV_CURRENT_YEAR
df = select (df,-c(MO_CREATED_DATE,SALES_BRANCH,REV_CURRENT_YEAR))

names(df)

 [1] "CUSTOMER"           "END_CUSTOMER"       "OFFER_PRICE"       
 [4] "SERVICE_LIST_PRICE" "MATERIAL_COST"      "SERVICE_COST"      
 [7] "PRICE_LIST"         "ISIC"               "SO_CREATED_DATE"   
[10] "TECH"               "OFFER_TYPE"         "BUSINESS_TYPE"     
[13] "COSTS_PRODUCT_A"    "COSTS_PRODUCT_B"    "COSTS_PRODUCT_C"   
[16] "OFFER_STATUS"       "COSTS_PRODUCT_D"    "COSTS_PRODUCT_E"   
[19] "SALES_LOCATION"     "TEST_SET_ID"        "COUNTRY_CODE"      
[22] "SALES_OFFICE"       "IDX_CUSTOMER"       "REV_CURRENT_YEAR.1"
[25] "REV_CURRENT_YEAR.2" "CREATION_YEAR"      "OWNERSHIP"         
[28] "CURRENCY"          


### 2.2 Process Binary Features

In [14]:
# Processed Features in 2.2:
#  * OFFER_STATUS         [Modified]
#  * END_CUSTOMER         [Deleted]
#  * HAS_END_CUSTOMER     [Created]
#  * ISIC                 [Deleted]
#  * HAS_ISIC             [Created]
#  * HAS_COSTS_PRODUCT_A  [Created]
#  * HAS_COSTS_PRODUCT_B  [Created]
#  * HAS_COSTS_PRODUCT_C  [Created]
#  * HAS_COSTS_PRODUCT_D  [Created]
#  * HAS_COSTS_PRODUCT_E  [Created]
#  * COUNTRY_CODE         [Deleted]
#  * IS_COUNTRY_CODE_CH   [Created]

In [15]:
%%R

# Manipulate "OFFER_STATUS" Feature: Replace string values with binary values
df = df %>%
  mutate(
    OFFER_STATUS = case_when(
      OFFER_STATUS %in% c("WIN","Win","Won","WON") ~ "1",
      OFFER_STATUS %in% c("LOsT","Lost","LOST","Lose") ~ "0",
    )
  )

In [16]:
%%R

# Manipulate "END_CUSTOMER" Feature: Convert to HAS_END_CUSTOMER
df = df %>%
  mutate(
    HAS_END_CUSTOMER = case_when(
      END_CUSTOMER %in% c(NA,"No") ~ 0,
      TRUE ~ 1 # Includes numbers and "Yes" values
      
    )
  )

# Delete column: END_CUSTOMER
df = select (df,-c(END_CUSTOMER))

In [17]:
%%R

# Manipulate "ISIC" Feature: Convert to HAS_ISIC
df = df %>%
  mutate(
    HAS_ISIC = case_when(
      ISIC %in% c(NA) ~ 0,
      TRUE ~ 1 # Includes numbers
    )
  )

# Delete column: ISIC
df = select (df,-c(ISIC))

In [18]:
%%R

# Manipulate "COUNTRY_CODE" Feature: Convert to IS_COUNTRY_CODE_CH
df = df %>%
  mutate(
    IS_COUNTRY_CODE_CH = case_when(
      COUNTRY_CODE %in% c("CH") ~ 1,
      COUNTRY_CODE %in% c("FR") ~ 0,
    )
  )

# Delete column: COUNTRY_CODE
df = select (df,-c(COUNTRY_CODE))


In [19]:
%%R

# Manipulate "COSTS_PRODUCT_*" Features: Convert to HAS_PRODUCT_*
df = df %>%
  mutate(
    HAS_COSTS_PRODUCT_A = case_when(
      COSTS_PRODUCT_A %in% c(0) ~ 0,
      TRUE ~ 1 # Includes floating point numbers
    ),
    HAS_COSTS_PRODUCT_B = case_when(
      COSTS_PRODUCT_B %in% c(0) ~ 0,
      TRUE ~ 1 # Includes floating point numbers
    ),
    HAS_COSTS_PRODUCT_C = case_when(
      COSTS_PRODUCT_C %in% c(0) ~ 0,
      TRUE ~ 1 # Includes floating point numbers
    ),
    HAS_COSTS_PRODUCT_D = case_when(
      COSTS_PRODUCT_D %in% c(0) ~ 0,
      TRUE ~ 1 # Includes floating point numbers
    ),
    HAS_COSTS_PRODUCT_E = case_when(
      COSTS_PRODUCT_E %in% c(0) ~ 0,
      TRUE ~ 1 # Includes floating point numbers
    )
  )

### 2.3 Process Other Features

In [20]:
# Processed Features in 2.3:
#  * TOTAL_COSTS_PRODUCT      [Created]
#  * COSTS_PRODUCT_A          [Deleted]
#  * COSTS_PRODUCT_B          [Deleted]
#  * COSTS_PRODUCT_C          [Deleted]
#  * COSTS_PRODUCT_D          [Deleted]
#  * COSTS_PRODUCT_E          [Deleted]
#  * CREATION_YEAR            [Modified]
#  * SINCE_CREATION_YEAR      [Created]
#  * REV_CURRENT_YEAR.1       [Modified]
#  * REV_CURRENT_YEAR.2       [Modified]
#  * REV_PERCENTAGE_INCREASE  [Created]
#  * OWNERSHIP_NO_INFO_AS_NA  [Created]
#  * OWNERSHIP_NA_AS_NO_INFO  [Created]
#  * SO_CREATED_DATE_SCALED   [Created]
#  * SO_CREATED_DATE          [Deleted]
#  * SO_CREATED_DATE_INTEGER  [Deleted]


In [21]:
%%R

# Create "TOTAL_COSTS_PRODUCT" Feature: Sum of "COSTS_PRODUCT_*"
df = df %>%
  mutate(
    TOTAL_COSTS_PRODUCT = COSTS_PRODUCT_A
    + COSTS_PRODUCT_B + 
      COSTS_PRODUCT_C + COSTS_PRODUCT_D + COSTS_PRODUCT_E
  )

# Delete columns: COSTS_PRODUCT_A to COSTS_PRODUCT_E
df = select (df,-c(COSTS_PRODUCT_A,COSTS_PRODUCT_B,
                                 COSTS_PRODUCT_C,COSTS_PRODUCT_D,
                                 COSTS_PRODUCT_E))

In [22]:
%%R

# Manipulate "CREATION_YEAR" Feature: Extract year
df = df %>%
  mutate(CREATION_YEAR = case_when(
    is.character(CREATION_YEAR) ~ CREATION_YEAR %>%
      substr(nchar(CREATION_YEAR) - 3, nchar(CREATION_YEAR)) %>%
      as.numeric()
  ))

# Create "SINCE_CREATION_YEAR" Feature: 2021 - CREATION_YEAR
df = df %>%
  mutate(SINCE_CREATION_YEAR = case_when(!is.na(CREATION_YEAR) ~ as.double(2021 - CREATION_YEAR)))

In [23]:
%%R

# Manipulate "REV_CURRENT_YEAR.1" and "REV_CURRENT_YEAR.2" Feature: Convert to EUR

# Convert negative values to zero
# df$REV_CURRENT_YEAR.1 = ifelse(df$REV_CURRENT_YEAR.1 < 0,0,df$REV_CURRENT_YEAR.1)
# df$REV_CURRENT_YEAR.2 = ifelse(df$REV_CURRENT_YEAR.2 < 0,0,df$REV_CURRENT_YEAR.2)

# Create "REV_PERCENTAGE_INCREASE" Feature: REV_CURRENT_YEAR.2 to REV_CURRENT_YEAR.1
# calculate_percentage_increase = function(new, old) {
#   100 * (new - old) / old
# }
# df = df %>%
#   mutate(REV_PERCENTAGE_INCREASE = case_when(
#     !is.na(REV_CURRENT_YEAR.1) & !is.na(REV_CURRENT_YEAR.2) ~
#       calculate_percentage_increase(REV_CURRENT_YEAR.1, REV_CURRENT_YEAR.2)
#   ))


# df = df %>% mutate( # If REV_CURRENT_YEAR.1 is 0 or null, fill it from REV_CURRENT_YEAR.2
#   REV_CURRENT_YEAR.1 = ifelse(
#     ((REV_CURRENT_YEAR.1 <= 0) |
#        is.na(REV_CURRENT_YEAR.1)) &
#       !is.na(REV_CURRENT_YEAR.2),
#     REV_CURRENT_YEAR.2,
#     REV_CURRENT_YEAR.1
#   ) %>% as.double()
# )
# 
# df = df %>% mutate( # If REV_CURRENT_YEAR.2 is 0 or null, fill it from REV_CURRENT_YEAR.1
#   REV_CURRENT_YEAR.2 = ifelse(
#     ((REV_CURRENT_YEAR.2 <= 0) |
#        is.na(REV_CURRENT_YEAR.2)) &
#       !is.na(REV_CURRENT_YEAR.1),
#     REV_CURRENT_YEAR.1,
#     REV_CURRENT_YEAR.2
#   ) %>% as.double()
# )


# Note: 2020 and 2021 annual average exchange rates are used.
# Source: https://www.x-rates.com/average/?from=USD&to=EUR&amount=1&year=2021
df = df %>%
  mutate(
    REV_CURRENT_YEAR.1 = case_when(
      CURRENCY ==  "Pound Sterling" ~ REV_CURRENT_YEAR.1 * 1.1438161149110808,
      CURRENCY ==  "Chinese Yuan" ~ REV_CURRENT_YEAR.1 * 0.12906362243502054,
      CURRENCY ==  "US Dollar" ~ REV_CURRENT_YEAR.1 * 0.8614249616963066,
      CURRENCY ==  "Euro" ~ REV_CURRENT_YEAR.1
    )
  )


#df = df %>%
#  mutate(
#    REV_CURRENT_YEAR.2 = case_when(
#      CURRENCY ==  "Pound Sterling" ~ REV_CURRENT_YEAR.2 * 1.1438161149110808,
#      CURRENCY ==  "Chinese Yuan" ~ REV_CURRENT_YEAR.2 * 0.12906362243502054,
#      CURRENCY ==  "US Dollar" ~ REV_CURRENT_YEAR.2 * 0.8614249616963066,
#      CURRENCY ==  "Euro" ~ REV_CURRENT_YEAR.2
#    )
#  )

# Create "REV_RATE" Feature: REV_CURRENT_YEAR.2 to REV_CURRENT_YEAR.1
df = df %>%
  mutate(REV_RATE = case_when(
    !is.na(REV_CURRENT_YEAR.1) & !is.na(REV_CURRENT_YEAR.2)  &  REV_CURRENT_YEAR.1 !=0 & REV_CURRENT_YEAR.2 !=0 ~
     REV_CURRENT_YEAR.1/REV_CURRENT_YEAR.2
  ))

# Create "REV_AVG" Feature: REV_CURRENT_YEAR.2 + REV_CURRENT_YEAR.1
df = df %>%
  mutate(REV_AVG = case_when(
    !is.na(REV_CURRENT_YEAR.1) & !is.na(REV_CURRENT_YEAR.2)  &  REV_CURRENT_YEAR.1 !=0 & REV_CURRENT_YEAR.2 !=0 ~
     (REV_CURRENT_YEAR.1+REV_CURRENT_YEAR.2)/2,
     !is.na(REV_CURRENT_YEAR.1) & !is.na(REV_CURRENT_YEAR.2)  &  (REV_CURRENT_YEAR.1 ==0 | REV_CURRENT_YEAR.2 ==0) ~
     (REV_CURRENT_YEAR.1+REV_CURRENT_YEAR.2),
      !is.na(REV_CURRENT_YEAR.1) & !is.na(REV_CURRENT_YEAR.2)  &  REV_CURRENT_YEAR.1 ==0 & REV_CURRENT_YEAR.2 ==0 ~
      0.00
  ))

In [24]:
%%R

# Create "OWNERSHIP_NO_INFO_AS_NA" Feature: Treat "No information" as NA value
df$OWNERSHIP_NO_INFO_AS_NA = ifelse(df$OWNERSHIP ==  "No information", NA, df$OWNERSHIP)

# Create "OWNERSHIP_NA_AS_NO_INFO" Feature: Treat NA values as "No information"
df$OWNERSHIP_NA_AS_NO_INFO = ifelse(is.na(df$OWNERSHIP), "No information", df$OWNERSHIP)


# Create "SO_CREATED_DATE_SCALED" Feature: Scale "SO_CREATED_DATE" x 100

df$SO_CREATED_DATE_INTEGER = as_datetime(df$SO_CREATED_DATE, format = "%d.%m.%Y %H:%M") # Parse date-format 1
date_format_2 = as_datetime(df$SO_CREATED_DATE, format = "%Y-%m-%d %H:%M:%S") # Parse date-format 1
df$SO_CREATED_DATE_INTEGER[is.na(df$SO_CREATED_DATE_INTEGER)] = date_format_2[!is.na(date_format_2)]

df$SO_CREATED_DATE_INTEGER = as.numeric(as.POSIXct(df$SO_CREATED_DATE_INTEGER))# Convert to Unix Time Stamp
standart_scale = function (x) (x - mean(x, na.rm = T)) / sd(x, na.rm = T)

df$SO_CREATED_DATE_SCALED = standart_scale(df$SO_CREATED_DATE_INTEGER) # Scale x 100

# Delete column: "SO_CREATED_DATE"
df = select (df,-c(SO_CREATED_DATE))

In [25]:
%%R

# FIXME: Delete this before submission.
write_csv(x = df, file = "interim_data/df_completed_1_2.csv")

## 3. Dealing with Extreme Values, Filling Missing Values and Data Splitting

### 3.1 Deal with Extreme Values in Numeric Features

In [26]:
%%R

df$TOTAL_COSTS_PRODUCT = ifelse(df$TOTAL_COSTS_PRODUCT < 0,-df$TOTAL_COSTS_PRODUCT,df$TOTAL_COSTS_PRODUCT)
df$TOTAL_COSTS_PRODUCT_LOG=log(df$TOTAL_COSTS_PRODUCT+1)

df$SERVICE_COST = ifelse(df$SERVICE_COST < 0,-df$SERVICE_COST,df$SERVICE_COST)
df$SERVICE_COST_LOG=log(df$SERVICE_COST+1)

df$OFFER_PRICE_LOG=log(df$OFFER_PRICE)

df$SERVICE_LIST_PRICE_LOG=log(df$SERVICE_LIST_PRICE+1)

df$MATERIAL_COST_LOG=log(df$MATERIAL_COST+1)

df$REV_CURRENT_YEAR_LOG.1=log(df$REV_CURRENT_YEAR.1+1)

df$REV_CURRENT_YEAR_LOG.2=log(df$REV_CURRENT_YEAR.2+1)

df$CREATION_YEAR_LOG=log(df$CREATION_YEAR)

df$SINCE_CREATION_YEAR_LOG=log(df$SINCE_CREATION_YEAR+1)

#For REV_PERCENTAGE_INCREASE
#Q1 <- quantile(df$REV_PERCENTAGE_INCREASE, .25,na.rm=T)
#Q3 <- quantile(df$REV_PERCENTAGE_INCREASE, .75,na.rm=T)
#IQR <- IQR(df$REV_PERCENTAGE_INCREASE,na.rm=T)
#df = df %>%  mutate(
#  REV_PERCENTAGE_INCREASE_NO_OUTLIER = case_when(
#    REV_PERCENTAGE_INCREASE < (Q1 - 3.0*IQR) ~ (Q1 - 3.0*IQR),
#    REV_PERCENTAGE_INCREASE > (Q3 + 3.0*IQR) ~ (Q3 + 3.0*IQR),
#    TRUE ~ REV_PERCENTAGE_INCREASE
#  )
#)

### 3.2 Deal with Extreme Values in Character Features

In [27]:
%%R

df$TECH_REDUCED_1 = ifelse(df$TECH %in% c("E", "EPS", "FP", "BP"), "E_EPS_FP_BP", df$TECH)
df$TECH_REDUCED_2_IS_F = ifelse(df$TECH == "F", 1, 0)

df$OFFER_TYPE_REDUCED_1 = ifelse(
  df$OFFER_TYPE %in% c(
    "FD",
    "EH",
    "FEI",
    "MSYS",
    "DCF",
    "GAM",
    "CP",
    "CS",
    "CI",
    "EN",
    "FIB",
    "PAT",
    "XCPS"
  ),
  "FD_EH_FEI_MSYS_DCF_GAM_CP_CS_CI_EN_FIB_PAT_XCPS",
  df$OFFER_TYPE
)
df$OFFER_TYPE_REDUCED_2 = ifelse(
  df$OFFER_TYPE %in% c(
    "FED",
    "CPP",
    "ED",
    "EV",
    "FD",
    "EH",
    "FEI",
    "MSYS",
    "DCF",
    "GAM",
    "CP",
    "CS",
    "CI",
    "EN",
    "FIB",
    "PAT",
    "XCPS"
  ),
  "FED_CPP_ED_EV_FD_EH_FEI_MSYS_DCF_GAM_CP_CS_CI_EN_FIB_PAT_XCPS",
  df$OFFER_TYPE
)

df$OWNERSHIP_NA_AS_NO_INFO_REDUCED = ifelse(
  df$OWNERSHIP_NA_AS_NO_INFO %in% c("Governmental", "Individual Person", "No information"),
  "Governmental_IndividualPerson_Noinformation",
  df$OWNERSHIP_NA_AS_NO_INFO
)

df$OWNERSHIP_NO_INFO_AS_NA_REDUCED = ifelse(
  df$OWNERSHIP_NO_INFO_AS_NA %in% c("Governmental", "Individual Person"),
  "Governmental_IndividualPerson",
  df$OWNERSHIP_NO_INFO_AS_NA
)

df$OWNERSHIP_REDUCED = ifelse(
  df$OWNERSHIP %in% c("Governmental", "Individual Person", "No information"),
  "Governmental_IndividualPerson_Noinformation",
  df$OWNERSHIP
)

df$SALES_OFFICE_REDUCED = ifelse(
  df$SALES_OFFICE %in% c(
    "Montpellier",
    "Monaco",
    "Limoges",
    "Vertical Market",
    "Others Functions"
  ),
  "Montpellier_Monaco_Limoges_Vertical Market_OthersFunctions",
  df$SALES_OFFICE
)

### 3.3 Create IS_NA Columns and Remove the Duplicate Features

In [28]:
%%R

# For numeric columns
#df$IS_NA_REV_PERCENTAGE_INCREASE = ifelse(is.na(df$REV_PERCENTAGE_INCREASE) == T,1,0)
df$IS_NA_REV_RATE = ifelse(is.na(df$REV_RATE) == T,1,0)
df$IS_NA_REV_AVG = ifelse(is.na(df$REV_AVG) == T,1,0)
df$IS_NA_REV_CURRENT_YEAR = ifelse(is.na(df$REV_CURRENT_YEAR.1) == T,1,0)


# For Categoric Columns
df$IS_NA_SALES_LOCATION = ifelse(is.na(df$SALES_LOCATION) == T,1,0)
df$IS_NA_SALES_OFFICE = ifelse(is.na(df$SALES_OFFICE) == T,1,0)
df$IS_NA_CURRENCY = ifelse(is.na(df$CURRENCY) == T,1,0)
df$IS_NA_OWNERSHIP_NO_INFO_AS_NA = ifelse(is.na(df$OWNERSHIP_NO_INFO_AS_NA) == T,1,0)


In [29]:
%%R

# FIXME: Delete this before submission.
write_csv(x = df, file = "interim_data/df_completed_1_2_3_with_mv_new.csv")

### 3.4 Fill Missing Values in Numeric Features

In [ ]:
%%R

# Replace with mean

df$REV_CURRENT_YEAR.1[is.na(df$REV_CURRENT_YEAR.1)] = mean(df$REV_CURRENT_YEAR.1, na.rm=TRUE)

df$REV_CURRENT_YEAR.2[is.na(df$REV_CURRENT_YEAR.2)] = mean(df$REV_CURRENT_YEAR.2, na.rm=TRUE)

df$CREATION_YEAR[is.na(df$CREATION_YEAR)] = mean(df$CREATION_YEAR, na.rm=TRUE)

df$SINCE_CREATION_YEAR[is.na(df$SINCE_CREATION_YEAR)] = mean(df$SINCE_CREATION_YEAR, na.rm=TRUE)

df$REV_CURRENT_YEAR_LOG.1[is.na(df$REV_CURRENT_YEAR_LOG.1)] = mean(df$REV_CURRENT_YEAR_LOG.1, na.rm=TRUE)

df$REV_CURRENT_YEAR_LOG.2[is.na(df$REV_CURRENT_YEAR_LOG.2)] = mean(df$REV_CURRENT_YEAR_LOG.2, na.rm=TRUE)

df$CREATION_YEAR_LOG[is.na(df$CREATION_YEAR_LOG)] = mean(df$CREATION_YEAR_LOG, na.rm=TRUE)

df$SINCE_CREATION_YEAR_LOG[is.na(df$SINCE_CREATION_YEAR_LOG)] = mean(df$SINCE_CREATION_YEAR_LOG, na.rm=TRUE)

df$REV_PERCENTAGE_INCREASE_NO_OUTLIER[is.na(df$REV_PERCENTAGE_INCREASE_NO_OUTLIER)] = mean(df$REV_PERCENTAGE_INCREASE_NO_OUTLIER, na.rm=TRUE)

df$REV_PERCENTAGE_INCREASE[is.na(df$REV_PERCENTAGE_INCREASE)] = mean(df$REV_PERCENTAGE_INCREASE, na.rm=TRUE)


### 3.5 Fill Missing Values in Character Features


In [ ]:
%%R

df$SALES_LOCATION[is.na(df$SALES_LOCATION)] = "Geneva West"

df$SALES_OFFICE[is.na(df$SALES_OFFICE)] = "Geneva"

df$SALES_OFFICE_REDUCED[is.na(df$SALES_OFFICE_REDUCED)] = "Geneva"

df$CURRENCY[is.na(df$CURRENCY)] = "NOT_GIVEN"

df$OWNERSHIP[is.na(df$OWNERSHIP)] = "NOT_GIVEN"

df$OWNERSHIP_NO_INFO_AS_NA[is.na(df$OWNERSHIP_NO_INFO_AS_NA)] = "NOT_GIVEN"

df$OWNERSHIP_NO_INFO_AS_NA_REDUCED[is.na(df$OWNERSHIP_NO_INFO_AS_NA_REDUCED)] = "NOT_GIVEN"

df$OWNERSHIP_REDUCED[is.na(df$OWNERSHIP_REDUCED)] = "NOT_GIVEN"


In [ ]:
%%R

# FIXME: Delete this before submission.
write_csv(x = df, file = "interim_data/df_completed_1_2_3.csv")

## 4. Feature Importance

In [ ]:
%%R

install.packages("party")
library(party)

In [ ]:
%%R

x = df

x = x[is.na(x$OFFER_STATUS) == F,]

x = x %>% select(-c(TEST_SET_ID,IDX_CUSTOMER,CUSTOMER))

In [ ]:
%%R

x$OFFER_STATUS = as.numeric(x$OFFER_STATUS)

a <- select_if(x, is.numeric)  
colnames(a)
cf1 <- cforest(a$OFFER_STATUS ~ . , data= a, control=cforest_unbiased(mtry=2,ntree=50)) # fit the random forest


In [ ]:
%%R

varimp(cf1) # get variable importance, based on mean decrease in accuracy

In [ ]:
%%R

#install.packages("earth")
library(earth)

marsModel <- earth(x$OFFER_STATUS ~ ., data=x) # build model
ev <- evimp (marsModel) # estimate variable importance

plot(ev)

In [ ]:
%%R

#install.packages("relaimpo")
library(relaimpo)
lmMod <- lm(OFFER_STATUS ~ . , data = x)  # fit lm() model
relImportance <- calc.relimp(lmMod, type = "lmg", rela = TRUE)  # calculate relative importance scaled to 100
sort(relImportance$lmg, decreasing=TRUE)  # relative importance

In [ ]:

names(boruta_output)

# Get significant variables including tentatives
boruta_signif <-
  getSelectedAttributes(boruta_output, withTentative = TRUE)
print(boruta_signif)

# Do a tentative rough fix
roughFixMod <- TentativeRoughFix(boruta_output)
boruta_signif <- getSelectedAttributes(roughFixMod)
print(boruta_signif)

# Variable Importance Scores
imps <- attStats(roughFixMod)
imps2 = imps[imps$decision != 'Rejected', c('meanImp', 'decision')]
head(imps2[order(-imps2$meanImp),])  # descending sort

x = summary(boruta_output$ImpHistory)
# Plot variable importance
plot(
  boruta_output,
  cex.axis = .7,
  las = 2,
  xlab = "",
  main = "Variable Importance"
)

In [ ]:
%%R

print("[INFO] Number of NA values in each column:")
for (i in 1:ncol(df)) {
  print(paste0(names(df)[i], ": ", sum(is.na(df[, i])), "/", nrow(df)))
}

In [ ]:
%%R

# fill in missing values
customers = customers %>% mutate(REV_CURRENT_YEAR.1 = ifelse(REV_CURRENT_YEAR.1 == 0,REV_CURRENT_YEAR.2, REV_CURRENT_YEAR.1))
customers = customers %>% mutate(REV_CURRENT_YEAR = ifelse(REV_CURRENT_YEAR == 0,REV_CURRENT_YEAR.2, REV_CURRENT_YEAR))

## Li - Section

In [ ]:
%%R
## check NA values in the labeled_data
#print("[INFO] Number of NA values in each column:")
#for (i in 1:ncol(labeled_data)) { # for-loop over columns
#  print(paste0(names(labeled_data)[i], ": ", sum(is.na(labeled_data[, i])), "/", nrow(labeled_data)))
#}

In [ ]:
%%R
#remove REV_CURRENT_YEAR column as it's the same as REV_CURRENT_YEAR.1
all_merged = subset(all_merged, select = -c(REV_CURRENT_YEAR))
#remove MO_CREATED_DATE column
all_merged = subset(all_merged, select = -c(MO_CREATED_DATE))

In [ ]:
%%R

#nice function from stackoverflow 
one_hot_encoding = function(df, columns="PRICE_LIST"){
  # create a copy of the original data.frame for not modifying the original
  df = cbind(df)
  # convert the columns to vector in case it is a string
  columns = c(columns)
  # for each variable perform the One hot encoding
  for (column in columns){
    unique_values = sort(unique(df[column])[,column])
    non_reference_values  = unique_values[c(-1)] # the first element is going 
                                                 # to be the reference by default
    for (value in non_reference_values){
      # the new dummy column name
      new_col_name = paste0(column,'_',value)
      # create new dummy column for each value of the non_reference_values
      df[new_col_name] <- with(df, ifelse(df[,column] == value, 1, 0))
    }
    # delete the one hot encoded column
    df[column] = NULL

  }
  return(df)
}

In [ ]:
%%R
#one_hot_encoding column PRICE_LIST, TECH, BUSINESS_TYPE, OFFER_TYPE
all_merged = one_hot_encoding(all_merged, c("PRICE_LIST"))
all_merged = one_hot_encoding(all_merged, c("TECH"))
all_merged = one_hot_encoding(all_merged, c("BUSINESS_TYPE"))
all_merged = one_hot_encoding(all_merged, c("OFFER_TYPE"))

In [ ]:
%%R
#Convert COUNTRY_CODE to binary
all_merged = all_merged %>%
  mutate(
    COUNTRY_CODE = case_when(
      COUNTRY_CODE %in% c("CH") ~ 0,
      COUNTRY_CODE %in% c("FR") ~ 1,
    ))

In [ ]:
%%R
glimpse(all_merged)
write_csv(x = all_merged, "interim_data/all_merged_3.csv")

In [ ]:
%%R

#install.packages("Hmisc")
#library(Hmisc)
#complete REV_CURRENT_YEAR.1 and REV_CURRENT_YEAR.2 by mean using impute method
#labeled_data$REV_CURRENT_YEAR.1 = impute(labeled_data$REV_CURRENT_YEAR.1, mean)
#labeled_data$REV_CURRENT_YEAR.2 = impute(labeled_data$REV_CURRENT_YEAR.2, mean)

In [ ]:
%%R

#print("[INFO] Number of NA values in each column:")
#for (i in 1:ncol(labeled_data)) { # for-loop over columns
#  print(paste0(names(labeled_data)[i], ": ", sum(is.na(labeled_data[, i])), "/", nrow(labeled_data)))
#}

In [ ]:
drive.flush_and_unmount()

## Saqib - Section

In [ ]:
%%R

all_merged = one_hot_encoding(all_merged, c("SALES_OFFICE"))
all_merged = one_hot_encoding(all_merged, c("SALES_BRANCH"))

glimpse(all_merged)

## Teofil - Section

In [ ]:
%%R

old_all_merged = all_merged

In [ ]:
%%R

customers = cst_df


cst_df = cst_df %>% mutate(REV_CURRENT_YEAR.1 = ifelse(REV_CURRENT_YEAR.1 == 0,REV_CURRENT_YEAR.2, REV_CURRENT_YEAR.1))
cst_df = cst_df %>% mutate(PREV_YEAR_PERCENTAGE_INCREASE.1 = ((REV_CURRENT_YEAR.1 - REV_CURRENT_YEAR.2)/REV_CURRENT_YEAR.2)*100)


################# REPLACEMENT BLOC FOR FINAL MERGE ########################
cst_df = cst_df %>% mutate(COUNTRY_CODE = case_when(
  COUNTRY == 'Switzerland' ~ "CH",
  COUNTRY == 'France' ~ "FR"
))
# Transform customer to integer
trnsc_geo_df$CUSTOMER <- as.numeric(trnsc_geo_df$CUSTOMER)
all_merged = left_join(trnsc_geo_df, cst_df, by = c("CUSTOMER", "COUNTRY_CODE"))
#################################################################################


########## SPLIT LABELED DATA INTO TRAIN AND VALIDATION BASED ON UNIQUE CUSTOMERS ##############
unique_customers = unique(labeled_data$CUSTOMER)
train_ids = sample(unique_customers, size= floor(0.8 * length(unique_customers)), replace=FALSE)

train_set = labeled_data %>% filter(CUSTOMER %in% train_ids)
validation_set = labeled_data %>%  filter(!CUSTOMER %in% train_ids)

In [1]:
!pip install missingno > /dev/null
!pip install category_encoders > /dev/null

In [2]:
#Import Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import missingno as msno
from sklearn.model_selection import train_test_split
from sklearn import base
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor, MLPClassifier
from category_encoders import BinaryEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,roc_auc_score,classification_report,confusion_matrix
from IPython.display import Image
import warnings
import pandas as pd
import numpy as np
import collections as c
import sklearn
import os

from sklearn.preprocessing import MultiLabelBinarizer
import category_encoders as ce
from sklearn import preprocessing

# use feature importance for feature selection
from numpy import loadtxt
from numpy import sort
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.feature_selection import SelectFromModel

# example of auto-sklearn for a classification dataset
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from autosklearn.classification import AutoSklearnClassifier
import autosklearn

import copy
import time
import pickle
import itertools
from typing import List
import datetime

%load_ext nb_black

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 1000)  # or None
pd.set_option("display.max_rows", 1000)  # or None
pd.set_option("display.max_colwidth", -1)  # or -1


/home/iceking/Desktop/TUM Lecture Docs/3. Business Analytics and Machine Learning (IN2028)/Analytics Cup/Project Files/venv/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex
/home/iceking/Desktop/TUM Lecture Docs/3. Business Analytics and Machine Learning (IN2028)/Analytics Cup/Project Files/venv/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


<IPython.core.display.Javascript object>

## Helper Functions

In [5]:
def type_separator(df: pd.DataFrame, print_results=False):
    dtype_names = ["categorical", "binary", "continuous", "integer", "numerical"]
    type_to_cols = dict.fromkeys(dtype_names, [])

    dtype_char_to_names = {
        "O": "categorical",
        "i": "integer",
        "u": "integer",
        "f": "continuous",
    }

    for col in df.columns:
        unique_vals = set(df[col].dropna().unique())

        if len(unique_vals) < 2 or (
            df[col].dtype == np.dtype("object") and len(unique_vals) > 500
        ):
            raise ValueError(
                f"[ERROR] Something wrong with column:{col} cannot be this case, check conversions!"
            )

        if unique_vals == {0, 1} or unique_vals == {0.0, 1.0}:
            type_to_cols["binary"] = type_to_cols["binary"] + [col]
        elif df[col].dtype.str[1] in dtype_char_to_names:
            dtype_char = df[col].dtype.str[1]
            dtype_name = dtype_char_to_names[dtype_char]
            type_to_cols[dtype_name] = type_to_cols[dtype_name] + [col]
        else:
            raise ValueError(
                f'[ERROR] Numpy data type:"{df[col].dtype}" of col:"{col}" not understood.'
            )

    type_to_cols["numerical"] = type_to_cols["continuous"] + type_to_cols["integer"]
    type_to_cols["nominal"] = type_to_cols["categorical"] + type_to_cols["binary"]
    for dtype_name, col_names in type_to_cols.items():
        assert len(type_to_cols[dtype_name]) == len(
            set(type_to_cols[dtype_name])
        ), f'For type:"{dtype_name}", some columns are duplicate in: {col_names}.'

    if print_results:
        for key, val in type_to_cols.items():
            print("type:", key, "columns:")
            for col in sorted(val):
                print(">", col)
            print("-" * 32)
    return type_to_cols


def get_null_columns(df: pd.DataFrame) -> List:
    return [col for col in df.columns if np.any(df[col].isna())]


def get_non_null_columns(df: pd.DataFrame) -> List:
    return [col for col in df.columns if not np.any(df[col].isna())]


def get_inf_columns(df: pd.DataFrame) -> List:
    return [col for col in df.columns if np.any(df[col] == np.inf)]


def print_nan_and_inf_columns(df: pd.DataFrame):
    print("NaN and Infinity Columns and Counts:")
    for col in sorted(get_null_columns(df)):
        print("> [NaN     ]", col, "Null Count:", np.sum(df[col].isna()))
    for col in sorted(get_inf_columns(df)):
        print("> [INFINITY]", col, "Inf Count:", np.sum(df[col] == np.inf))


def get_labeled_set(df: pd.DataFrame, target_col: str = "OFFER_STATUS"):
    return df[~np.isnan(df[target_col])]


def get_unlabeled_set(df: pd.DataFrame, target_col: str = "OFFER_STATUS"):
    return df[np.isnan(df[target_col])]


def MissingUniqueStatistics(df, show_unique_values=False):

    total_entry_list = []
    total_missing_value_list = []
    missing_value_ratio_list = []
    data_type_list = []
    unique_values_list = []
    number_of_unique_values_list = []
    variable_name_list = []

    for col in df.columns:

        variable_name_list.append(col)
        missing_value_ratio = round((df[col].isna().sum() / len(df[col])), 4)
        total_entry_list.append(df[col].shape[0] - df[col].isna().sum())
        total_missing_value_list.append(df[col].isna().sum())
        missing_value_ratio_list.append(missing_value_ratio)
        data_type_list.append(df[col].dtype)
        unique_values_list.append(list(df[col].unique()))
        number_of_unique_values_list.append(len(df[col].unique()))

    data_info_df = pd.DataFrame(
        {
            "Variable": variable_name_list,
            "#_Total_Entry": total_entry_list,
            "#_Missing_Value": total_missing_value_list,
            "%_Missing_Value": missing_value_ratio_list,
            "Data_Type": data_type_list,
            "Unique_Values": unique_values_list,
            "#_Uniques_Values": number_of_unique_values_list,
        }
    )
    if not show_unique_values:
        data_info_df = data_info_df.drop("Unique_Values", axis=1)

    return data_info_df.sort_values(by="#_Missing_Value", ascending=False).set_index(
        "Variable"
    )


def histogram(df, feature):  # Histogram of the target categories
    %matplotlib inline
    ncount = len(df)
    ax = sns.countplot(x=feature, data=df, palette="hls")
    sns.set(font_scale=1)
    ax.set_xlabel("Target Segments")
    plt.xticks(rotation=90)
    ax.set_ylabel("Number of Observations")
    fig = plt.gcf()
    fig.set_size_inches(12, 5)
    # Make twin axis
    ax2 = ax.twinx()
    # Switch so count axis is on right, frequency on left
    ax2.yaxis.tick_left()
    ax.yaxis.tick_right()
    # Also switch the labels over
    ax.yaxis.set_label_position("right")
    ax2.yaxis.set_label_position("left")
    ax2.set_ylabel("Frequency [%]")
    for p in ax.patches:
        x = p.get_bbox().get_points()[:, 0]
        y = p.get_bbox().get_points()[1, 1]
        ax.annotate(
            "{:.2f}%".format(100.0 * y / ncount),
            (x.mean(), y),
            ha="center",
            va="bottom",
        )  # set the alignment of the text
    # Use a LinearLocator to ensure the correct number of ticks
    ax.yaxis.set_major_locator(ticker.LinearLocator(11))
    # Fix the frequency range to 0-100
    ax2.set_ylim(0, 100)
    ax.set_ylim(0, ncount)
    # And use a MultipleLocator to ensure a tick spacing of 10
    ax2.yaxis.set_major_locator(ticker.MultipleLocator(10))
    # Need to turn the grid on ax2 off, otherwise the gridlines end up on top of the bars
    ax2.grid(None)
    plt.title("Histogram of Binary Target Categories", fontsize=20, y=1.08)
    plt.show()
    plt.savefig("target_histogram.png")
    del ncount, x, y

    # USAGE: histogram(data, "CLASS")

<IPython.core.display.Javascript object>

## From Submission 3, With H2O

In [35]:
def apply_h2o(data, max_runtime_secs=10 * 60, max_models=20, balance_classes=True,
              models_dir_path = './h2o_models_with_data',training_with_all=False):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
    X_train, X_test, y_train, y_test = data
    
    from h2o.automl import H2OAutoML
    import h2o
    h2o.init(max_mem_size="32G")
    
    if not os.path.exists(models_dir_path):
        os.makedirs(models_dir_path)
        
    results_dir_path =f'{models_dir_path}/results_ts_{timestamp}'
    os.makedirs(results_dir_path)
        
    train_path =f"{results_dir_path}/train.csv"
    test_path =f"{results_dir_path}/test.csv"

    pd.concat([X_train, Y_train], axis=1).to_csv(train_path, index=False, header=True)
    pd.concat([X_test, Y_test], axis=1).to_csv(test_path, index=False, header=True)

    train = h2o.import_file(train_path)
    test = h2o.import_file(test_path)

    h2o.init(max_mem_size="36G")

    x = train.columns
    train["OFFER_STATUS"] = train["OFFER_STATUS"].asfactor()
    x.remove("OFFER_STATUS")
    aml = H2OAutoML(
        max_models=max_models,
        balance_classes=balance_classes,
        max_runtime_secs=int(max_runtime_secs),
        seed=42,
    )
    if  training_with_all:
        saved_model_path=f'{results_dir_path}/model_withALL'
        print(f'[INFO] Model will be saved here:"{saved_model_path}"',)
    aml.train(x=x, y="OFFER_STATUS", training_frame=train)
    
    print("[INFO] Timestamp:",timestamp)
    print('[INFO] AML Leaderboard',aml.leaderboard)
    
    model_bac_scores = []
    
    if  training_with_all:
        saved_model_path=f'{results_dir_path}/model_withALL'
        h2o.save_model(model=aml.leader, path=saved_model_path, force=True)
    else:
        for i in range(int(aml.max_models * 1.5)):
            current_model = aml.leaderboard[i, 0]
            if current_model == "NA":
                print(f"[INFO] Found {i} models in total.")
                break
            current_model = h2o.get_model(current_model)
            new_pred = current_model.predict(test)
            new_pred = new_pred[0].as_data_frame().values.flatten()
            model_bac_score = balanced_accuracy_score(Y_test, new_pred)
            print(f'[INFO] Model #{i}, BAC={model_bac_score}.')
            model_bac_scores.append(model_bac_score)

        index_max_bac_score = model_bac_scores.index(max(model_bac_scores))
    
        saved_model_path=f'{results_dir_path}/model_{"bac_%.3f" % model_bac_scores[index_max_bac_score]}'
        h2o.save_model(model=h2o.get_model(aml.leaderboard[index_max_bac_score, 0]), 
                   path=saved_model_path, force=True)
    
    print("[INFO] RESULTS:")
    print(f' > Train data saved to:"{train_path}".')
    print(f' > Test data saved to :"{test_path}".')
    print(f' > H20 Model saved to :"{saved_model_path}".')

    return aml, model_bac_scores


<IPython.core.display.Javascript object>

In [ ]:
def code_block(params:dict, verbose=False, calc_feature_importances=False):
    
    ####### Get Parameters - Start ################
    df = pd.read_csv(params['preprocessed_data_path']) # Read Data
    ###############################################
    
    ####### Block Specific Helper Functions #######
    def print_if_verbose(*args, **kwargs):
        if verbose:
            print(*args, **kwargs)
    ###############################################

    ###############################################
    id_columns = ["CUSTOMER", "TEST_SET_ID", "IDX_CUSTOMER"]
    unnecessary_reduced_cols = [
        "OFFER_TYPE_REDUCED_1",
        "OFFER_TYPE_REDUCED_2",
        "SALES_OFFICE_REDUCED",
    ]

    to_be_dropped_cols = id_columns + unnecessary_reduced_cols
    df = df.drop(to_be_dropped_cols, axis=1)

    # new columns
    df["ADDITIONAL_COST"] = df["OFFER_PRICE"] - df["MATERIAL_COST"] - df["SERVICE_COST"]
    df["TOTAL_COST"] = df["MATERIAL_COST"] + df["SERVICE_COST"]

    ###############################################

    assert 'OFFER_STATUS' not in type_separator(df)["categorical"]
    for col in type_separator(df)["categorical"]:
        num_unq = len(df[col].unique())
        trimmed_col = col.strip().replace(" ", "_")
        if num_unq < 5:
            print_if_verbose(f"[INFO] Col:{col},num_of_unq:{num_unq}, applying 1-HOT encoding.")
            onehot_df = pd.get_dummies(df[col])
            onehot_df = onehot_df.add_prefix(trimmed_col + "_1HOTENC_")
            df = pd.concat((df, onehot_df), axis=1)
        elif num_unq >= 5:
            print_if_verbose(f"[INFO] Col:{col},num_of_unq:{num_unq}, applying BINARY encoding.")
            encoder = ce.BinaryEncoder(cols=[col])
            binenc_df = encoder.fit_transform(df[[col]])
            binenc_df.columns = [
                f"{trimmed_col}_BINENC_{i}" for i in range(len(binenc_df.columns))
            ]
            df = pd.concat((df, binenc_df), axis=1)

    for col in type_separator(df)["categorical"]:
        if col != 'OFFER_STATUS':
            df[col] = pd.Categorical(df[col])

    ##########################

    ###### ADDITIONAL < FEATURES - START
    add_less_than_features = True
    print("[WARN] Adding Less Than Features:", add_less_than_features)

    if add_less_than_features:
        raw_num_cols = type_separator(df)["numerical"]
        nonraw_strings_in_cols = ("OFFER_STATUS","IS_NA","HAS_","1HOTENC","BINENC","IS_","_LOG")
        raw_num_cols = [a for a in raw_num_cols if not any( x in a for x in nonraw_strings_in_cols)]

        raw_numeric_cols_combinations = list(itertools.combinations(raw_num_cols, r=2))
        new_less_than_cols = []
        for col1, col2 in raw_numeric_cols_combinations:
            new_col = np.where(
                np.isnan(df[col1]) | np.isnan(df[col2]), np.nan, (df[col1] < df[col2])
            )
            name_of_new_col = col1 + "_<_" + col2

            if (len(np.unique(new_col[~np.isnan(new_col)])) > 1):  # Not all values are 1 or 0
                new_less_than_cols.append(pd.Series(new_col, name=name_of_new_col))
                print_if_verbose(f'[INFO] Added new "less than" column: "{name_of_new_col}".')
            else:
                print_if_verbose(f'[INFO] NOT added "Less Then" column: "{name_of_new_col}".')

        df = pd.concat([df, pd.concat(new_less_than_cols, axis=1)], axis=1)

    ############# DROPPP -START
    drop_cols = [
        ############## Correlation are same or very similar
        "OWNERSHIP_NA_AS_NO_INFO_1HOTENC_Privately Owned/Publicly Traded",
        "OWNERSHIP_NA_AS_NO_INFO_REDUCED_1HOTENC_Privately Owned/Publicly Traded",
        "OWNERSHIP_NO_INFO_AS_NA_1HOTENC_Privately Owned/Publicly Traded",
        "OWNERSHIP_NO_INFO_AS_NA_REDUCED_1HOTENC_Privately Owned/Publicly Traded",
        "OWNERSHIP_NA_AS_NO_INFO_1HOTENC_Individual Person",
        "OWNERSHIP_NA_AS_NO_INFO_1HOTENC_Governmental",
        "OWNERSHIP_NA_AS_NO_INFO_1HOTENC_No information",
        "OWNERSHIP_NO_INFO_AS_NA_REDUCED_1HOTENC_NOT_GIVEN",
        "OWNERSHIP_NO_INFO_AS_NA_1HOTENC_NOT_GIVEN",
        "OWNERSHIP_REDUCED_1HOTENC_NOT_GIVEN",
        "TECH_REDUCED_2_IS_F",
        "TECH_BINENC_0",
        "SINCE_CREATION_YEAR_<_REV_PERCENTAGE_INCREASE_NO_OUTLIER",  # (34, False)
        ############## Experimentally
        "TOTAL_COSTS_PRODUCT_LOG",
        "CURRENCY_BINENC_0",  # (19, False)
        "OWNERSHIP_BINENC_0",  # (20, False)
        "OWNERSHIP_NO_INFO_AS_NA_1HOTENC_Individual Person",  # (21, False)
        "IS_NA_SALES_LOCATION",  # (22, False)
        "IS_NA_SALES_OFFICE",  # (23, False)
        ## From XGB Feature Importance
        "SERVICE_COST_<_CREATION_YEAR",  # 0.0
        "REV_CURRENT_YEAR.1_<_REV_PERCENTAGE_INCREASE",  # 0.0
        "CREATION_YEAR_<_TOTAL_COST",  # 0.0
        "CREATION_YEAR_<_REV_PERCENTAGE_INCREASE",  # 0.0
        "CREATION_YEAR_<_ADDITIONAL_COST",  # 0.0
    ]

    for col in drop_cols:
        if col in list(df.columns):
            print_if_verbose("[INFO] Dropped:", col)
            df = df.drop(col, axis=1)

    if calc_feature_importances:
        def calculate_feature_importances():
            from boruta import BorutaPy
            from sklearn.ensemble import RandomForestClassifier

            df_new = df[~np.isnan(df["OFFER_STATUS"])].copy()
            columns_to_be_dropped = get_null_columns(df_new)
            if "categorical" in type_separator(df_new):
                columns_to_be_dropped += [
                    col for col in type_separator(df_new)["categorical"]
                    if col in df_new.columns
                ]
            columns_to_be_dropped = set(columns_to_be_dropped)
            if "OFFER_STATUS" in columns_to_be_dropped:
                columns_to_be_dropped.remove("OFFER_STATUS")

            df_new = df_new.drop(columns_to_be_dropped, axis=1)
            X, y = df_new.drop("OFFER_STATUS", axis=1), df_new["OFFER_STATUS"]

            forest = RandomForestClassifier(n_jobs=-1, class_weight="balanced", max_depth=5)
            forest.fit(X, y)
            
            feat_selector = BorutaPy( # define Boruta feature selection method
                forest, n_estimators="auto", verbose=2,
                random_state=42,
            )
            
            feat_selector.fit(X.to_numpy(), y.to_numpy()) # find all relevant features
            feature_importances = list(zip(feat_selector.ranking_,feat_selector.support_,X.columns))
            for item in sorted(feature_importances):
                print(item, ",")
        calculate_feature_importances()

    df_for_test = df[np.isnan(df["OFFER_STATUS"])]
    df = df[~np.isnan(df["OFFER_STATUS"])]
    
    X, y = df.drop("OFFER_STATUS", axis=1), df["OFFER_STATUS"] # Col Selection & Conversion
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.33, random_state=42)
    
    if params['classifer'] == 'h2o':
        return apply_h2o(data=[X_train, X_test, y_train, y_test],max_runtime_secs=params['max_runtime_secs'])


In [ ]:
result = code_block(
    params={
        "max_runtime_secs": 3 * 60,
        "preprocessed_data_path": "interim_data/df_completed_1_2_3_with_mv_new.csv",
        # "preprocessed_data_path": "interim_data/df_completed_1_2_3_with_mv.csv",
        "classifer": "h2o",
    }
)
result

## From Submission 4, With H2O & AutoSklearn

In [9]:
# def code_block(params:dict, verbose=False, calc_feature_importances=False):
# "max_runtime_secs": 3 * 60,
verbose = False
training_mode = False  # Make this false when predicting the real test set
is_data_analysis_and_visualization_part = False
preprocessed_data_path = "interim_data/df_completed_1_2_3_with_mv_new.csv"
test_size_in_split = 0.33

<IPython.core.display.Javascript object>

In [10]:
####### Block Specific Helper Functions #######
def print_if_verbose(*args, **kwargs):
    if verbose:
        print(*args, **kwargs)


###############################################

############# Data Postprocessing #############
df = pd.read_csv(preprocessed_data_path).drop(  # Read Data
    ["CUSTOMER", "TEST_SET_ID", "IDX_CUSTOMER"], axis=1
)

# Add some new columns
df["ADDITIONAL_COST"] = df["OFFER_PRICE"] - df["MATERIAL_COST"] - df["SERVICE_COST"]
df["TOTAL_COST"] = df["MATERIAL_COST"] + df["SERVICE_COST"]
df["TOTAL_COST_LOG"] = df["MATERIAL_COST"] + df["SERVICE_COST"]
df["TOTAL_COST_RATIO"] = df["MATERIAL_COST"] / df["TOTAL_COST"]

# HIGH ABSOLUTE CORRELATIONS BETWEEN INDEPENDENT VALUES:
# > abs(Corr)=1.00 with: "SO_CREATED_DATE_SCALED"    <->   "SO_CREATED_DATE_INTEGER"
# > abs(Corr)=1.00 with: "SINCE_CREATION_YEAR"       <->   "CREATION_YEAR_LOG"
# > abs(Corr)=1.00 with: "SINCE_CREATION_YEAR"       <->   "CREATION_YEAR"
# > abs(Corr)=1.00 with: "IS_NA_REV_CURRENT_YEAR"    <->   "IS_NA_CURRENCY"
# > abs(Corr)=1.00 with: "MATERIAL_COST_LOG"         <->   "SERVICE_LIST_PRICE_LOG"
# > abs(Corr)=1.00 with: "TOTAL_COST"                <->   "TOTAL_COST_LOG"
# > abs(Corr)=0.99 with: "TOTAL_COST"                <->   "IS_NA_REV_AVG"
# > abs(Corr)=0.99 with: "SERVICE_LIST_PRICE"        <->   "MATERIAL_COST"
# > abs(Corr)=0.99 with: "REV_AVG"                   <->   "REV_CURRENT_YEAR.2"
# > abs(Corr)=0.99 with: "REV_AVG"                   <->   "REV_CURRENT_YEAR.1"
# > abs(Corr)=0.99 with: "IS_NA_SALES_OFFICE"        <->   "IS_NA_SALES_LOCATION"
# > abs(Corr)=0.99 with: "IS_NA_REV_CURRENT_YEAR"    <->   "IS_NA_OWNERSHIP_NO_INFO_AS_NA"
# > abs(Corr)=0.96 with: "TOTAL_COST"                <->   "OFFER_PRICE"
# > abs(Corr)=0.96 with: "SINCE_CREATION_YEAR_LOG"   <->   "SINCE_CREATION_YEAR"

selected_cols_by_boruta = [
    "OFFER_STATUS",  ################ Target Column
    "ADDITIONAL_COST",  # , # 1, # True',
    "BUSINESS_TYPE",  # , # 1, # True',
    "OFFER_TYPE",  # , # 1, # True',
    "OFFER_TYPE_REDUCED_1",  # , # 1, # True',
    "OFFER_TYPE_REDUCED_2",  # , # 1, # True',
    "REV_AVG",  # , # 1, # True',
    "REV_CURRENT_YEAR.1",  # , # 1, # True',
    "REV_CURRENT_YEAR_LOG.1",  # , # 1, # True',
    "REV_CURRENT_YEAR_LOG.2",  # , # 1, # True',
    "REV_RATE",  # , # 1, # True',
    "SALES_LOCATION",  # , # 1, # True',
    "SALES_OFFICE",  # , # 1, # True',
    "SALES_OFFICE_REDUCED",  # , # 1, # True',
    "SERVICE_COST",  # , # 1, # True',
    "SERVICE_COST_LOG",  # , # 1, # True',
    "SO_CREATED_DATE_INTEGER",  # , # 1, # True',
    # "SO_CREATED_DATE_SCALED",  # , # 1, # True',
    # "TOTAL_COST_RATIO",  # , # 1, # True',
    "OFFER_PRICE_LOG",  # , # 2, # False',
    "OFFER_PRICE",  # , # 3, # False',
    "TECH_REDUCED_1",  # , # 4, # False',
    # "REV_CURRENT_YEAR.2",  # , # 5, # False',
    # "TOTAL_COST",  # , # 5, # False',
    "TECH",  # , # 7, # False',
    "TOTAL_COST_LOG",  # , # 7, # False',
    # "PRICE_LIST",  # , # 9, # False',
    "SERVICE_LIST_PRICE",  # , # 9, # False',
    "SERVICE_LIST_PRICE_LOG",  # , # 11, # False',
    # "MATERIAL_COST_LOG",  # , # 12, # False',
    "MATERIAL_COST",  # , # 13, # False',
    "CURRENCY",  # , # 14, # False',
    # "SINCE_CREATION_YEAR",  # , # 15, # False',
    # "CREATION_YEAR",  # , # 16, # False',
    # "CREATION_YEAR_LOG",  # , # 16, # False',
    # "SINCE_CREATION_YEAR_LOG",  # , # 18, # False',
    # 'TECH_REDUCED_2_IS_F', # , # 19, # False',
    # 'OWNERSHIP_NA_AS_NO_INFO', # , # 21, # False',
    # 'TOTAL_COSTS_PRODUCT', # , # 21, # False',
    # 'TOTAL_COSTS_PRODUCT_LOG', # , # 21, # False',
    # 'OWNERSHIP_NA_AS_NO_INFO_REDUCED', # , # 23, # False',
    # 'OWNERSHIP', # , # 24, # False',
    # 'OWNERSHIP_REDUCED', # , # 24, # False',
    # 'OWNERSHIP_NO_INFO_AS_NA_REDUCED', # , # 26, # False',
    # 'OWNERSHIP_NO_INFO_AS_NA', # , # 27, # False',
    # 'IS_COUNTRY_CODE_CH', # , # 28, # False',
    # 'HAS_END_CUSTOMER', # , # 29, # False',
    # 'IS_NA_REV_RATE', # , # 30, # False',
    # 'IS_NA_REV_AVG', # , # 31, # False',
    # 'HAS_COSTS_PRODUCT_D', # , # 32, # False',
    # 'HAS_COSTS_PRODUCT_B', # , # 33, # False',
    # 'IS_NA_REV_CURRENT_YEAR', # , # 33, # False',
    # 'IS_NA_CURRENCY', # , # 35, # False',
    # 'IS_NA_OWNERSHIP_NO_INFO_AS_NA', # , # 35, # False',
    # 'HAS_ISIC', # , # 37, # False',
    # 'HAS_COSTS_PRODUCT_A', # , # 38, # False',
    # 'HAS_COSTS_PRODUCT_E', # , # 39, # False',
    # 'HAS_COSTS_PRODUCT_C', # , # 40, # False',
    # 'IS_NA_SALES_OFFICE', # , # 41, # False',
    # 'IS_NA_SALES_LOCATION', # , # 42, # False'
]
assert "OFFER_STATUS" in selected_cols_by_boruta
df = df[selected_cols_by_boruta]
###############################################

######## Data Analysis and Visualization ######
if is_data_analysis_and_visualization_part:

    def data_analysis_and_visualization_part():
        print("[INFO] Correlation Matrix:")
        f, ax = plt.subplots(figsize=(30, 13))
        sns.heatmap(abs(df.corr()), annot=True)
        plt.show()

        print("[INFO] Missing Values:")
        msno.bar(df, color="#79ccb3", sort="descending")
        plt.show()

        print("[INFO] Missing Value Correlations - 1:")
        msno.matrix(
            df[get_null_columns(df)],
            color=(0.45, 0.45, 0.64),
            figsize=(27, 10),
            width_ratios=(10, 0),
        )
        plt.show()

        print("[INFO] Missing Value Correlations - 2:")
        msno.dendrogram(df[get_null_columns(df)])
        plt.show()

        print("[INFO] Missing Value Correlations - 3:")
        msno.heatmap(df[get_null_columns(df)])
        plt.show()

    data_analysis_and_visualization_part()
###############################################

######## Outlier Detection #############
print("[INFO] Outlier Detection has begun.")


col_to_type = [
    ######################### Nominal
    ("BUSINESS_TYPE", "Nominal"),
    ("CURRENCY", "Nominal"),
    ("HAS_COSTS_PRODUCT_A", "Nominal"),
    ("HAS_COSTS_PRODUCT_B", "Nominal"),
    ("HAS_COSTS_PRODUCT_C", "Nominal"),
    ("HAS_COSTS_PRODUCT_D", "Nominal"),
    ("HAS_COSTS_PRODUCT_E", "Nominal"),
    ("HAS_END_CUSTOMER", "Nominal"),
    ("HAS_ISIC", "Nominal"),
    ("IS_COUNTRY_CODE_CH", "Nominal"),
    ("IS_NA_CURRENCY", "Nominal"),
    ("IS_NA_OWNERSHIP_NO_INFO_AS_NA", "Nominal"),
    ("IS_NA_REV_CURRENT_YEAR", "Nominal"),
    ("IS_NA_REV_AVG", "Nominal"),
    ("IS_NA_REV_RATE", "Nominal"),
    ("IS_NA_SALES_OFFICE", "Nominal"),
    ("IS_NA_SALES_LOCATION", "Nominal"),
    ("OFFER_STATUS", "Nominal"),
    ("OFFER_TYPE", "Nominal"),
    ("OFFER_TYPE_REDUCED_1", "Nominal"),
    ("OFFER_TYPE_REDUCED_2", "Nominal"),
    ("OWNERSHIP", "Nominal"),
    ("OWNERSHIP_NA_AS_NO_INFO", "Nominal"),
    ("OWNERSHIP_NA_AS_NO_INFO_REDUCED", "Nominal"),
    ("OWNERSHIP_NO_INFO_AS_NA", "Nominal"),
    ("OWNERSHIP_NO_INFO_AS_NA_REDUCED", "Nominal"),
    ("OWNERSHIP_REDUCED", "Nominal"),
    ("PRICE_LIST", "Nominal"),
    ("SALES_LOCATION", "Nominal"),
    ("SALES_OFFICE", "Nominal"),
    ("SALES_OFFICE_REDUCED", "Nominal"),
    ("TECH", "Nominal"),
    ("TECH_REDUCED_1", "Nominal"),
    ("TECH_REDUCED_2_IS_F", "Nominal"),
    ######################### Continuous
    ("CREATION_YEAR", "Continuous"),
    ("CREATION_YEAR_LOG", "Continuous"),
    ("MATERIAL_COST", "Continuous"),
    ("MATERIAL_COST_LOG", "Continuous"),
    ("OFFER_PRICE", "Continuous"),
    ("OFFER_PRICE_LOG", "Continuous"),
    ("REV_AVG", "Continuous"),
    ("REV_CURRENT_YEAR.1", "Continuous"),
    ("REV_CURRENT_YEAR_LOG.1", "Continuous"),
    ("REV_CURRENT_YEAR.2", "Continuous"),
    ("REV_CURRENT_YEAR_LOG.2", "Continuous"),
    ("REV_RATE", "Continuous"),
    ("SERVICE_COST", "Continuous"),
    ("SERVICE_COST_LOG", "Continuous"),
    ("SERVICE_LIST_PRICE", "Continuous"),
    ("SERVICE_LIST_PRICE_LOG", "Continuous"),
    ("SINCE_CREATION_YEAR", "Continuous"),
    ("SINCE_CREATION_YEAR_LOG", "Continuous"),
    ("SO_CREATED_DATE_INTEGER", "Continuous"),
    ("SO_CREATED_DATE_SCALED", "Continuous"),
    ("TOTAL_COST", "Continuous"),
    ("TOTAL_COST_LOG", "Continuous"),
    ("TOTAL_COST_RATIO", "Continuous"),
    ("TOTAL_COSTS_PRODUCT", "Continuous"),
    ("TOTAL_COSTS_PRODUCT_LOG", "Continuous"),
    ("ADDITIONAL_COST", "Continuous"),
]
df_info = MissingUniqueStatistics(df)
df_col_to_type = pd.DataFrame(col_to_type, columns=["Variable", "Variable_Type"])
df_info = df_info.merge(df_col_to_type, on="Variable").set_index("Variable")
print_if_verbose(df_info)

numerical_columns = list(
    df_info.loc[
        (df_info.loc[:, "Variable_Type"] == "Cardinal")
        | (df_info.loc[:, "Variable_Type"] == "Continuous")
    ].index
)
assert len(numerical_columns) > 0

categorical_columns = list(
    df_info.loc[
        (df_info.loc[:, "Variable_Type"] == "Nominal")
        | (df_info.loc[:, "Variable_Type"] == "Ordinal")
    ].index
)
assert len(categorical_columns) > 0


assert len({x[0] for x in col_to_type}) == len(
    [x[0] for x in col_to_type]
), "There are some duplicate columns in col_to_type list!"
assert not np.any(df_info.isna()), "Some variable types are null!"
assert len(df_info.index) == df_info.shape[0], "Some Rows are not unique!"

df_all_processed = df.copy()
df_for_unlabeled_set = df_all_processed[np.isnan(df_all_processed["OFFER_STATUS"])]
df = df_all_processed[~np.isnan(df["OFFER_STATUS"])]

if training_mode:
    x, y = df.drop(["OFFER_STATUS"], axis=1), df["OFFER_STATUS"]
    X_train, X_test, Y_train, Y_test = train_test_split(
        x, y, test_size=test_size_in_split, stratify=y, random_state=42
    )
    if verbose:
        histogram(X_train, Y_train)
        histogram(X_test, Y_test)
else:
    X_train, Y_train = df.drop(["OFFER_STATUS"], axis=1), df["OFFER_STATUS"]

    X_test = df_for_unlabeled_set.drop(["OFFER_STATUS"], axis=1)
    Y_test = df_for_unlabeled_set["OFFER_STATUS"]


print_if_verbose(
    f"df_all_processed:{df_all_processed.shape}",
    f"df_for_unlabeled_set:{df_for_unlabeled_set.shape}",
    f"X_train:{X_train.shape}",
    f"Y_train:{Y_train.shape}",
    f"X_test:{X_test.shape}",
    f"Y_test:{Y_test.shape}",
    sep="\n",
)

[INFO] Outlier Detection has begun.


<IPython.core.display.Javascript object>

In [12]:
### Finding sparse columns

sparse_columns = []
for col in numerical_columns:
    if df[col].quantile(0.01) == df[col].quantile(0.25) == df[col].mode()[0]:
        sparse_columns.append(col)

sparse_columns_2 = []
for col in numerical_columns:
    if df[col].quantile(0.01) == df[col].quantile(0.25):
        sparse_columns_2.append(col)
assert sparse_columns == sparse_columns_2

print("Sparse Columns:", sparse_columns)


def HardEdgeReduction(
    df, numerical_columns, sparse_columns, upper_quantile=0.99, lower_quantile=0.01
):
    """
    Algorithm 'HER(Hard-Edges Method)' applies induction to the elements of a value line which are:

        - lower than the 1th quantile to that quantile and
        - upper than the 99th quantile to that quantile.

    Main aim is to diminish negative effects of outlier values on analytical operations being performed.
    """

    import pandas as pd
    import psutil, os, gc, time

    print_if_verbose("HardEdgeReduction process has began:\n")
    proc = psutil.Process(os.getpid())
    gc.collect()
    mem_0 = proc.memory_info().rss
    start_time = time.time()

    epsilon = 0.0001  # for zero divisions

    # Define boundaries that we will use for Reduction operation
    df_outlier_cleaned = df.copy()

    print_if_verbose(
        "Detected outliers will be replaced with edged quantiles/percentiles: 1% and 99%!\n"
    )
    print_if_verbose("Total number of rows is: %s\n" % df_outlier_cleaned.shape[0])

    outlier_boundries_dict = {}

    for col in numerical_columns:

        if col in sparse_columns:
            nonsparse_data = pd.DataFrame(
                df_outlier_cleaned[
                    df_outlier_cleaned[col] != df_outlier_cleaned[col].mode()[0]
                ][col]
            )

            if (  # For lower threshold (left-hand-side)
                nonsparse_data[col].quantile(lower_quantile)
                < df_outlier_cleaned[col].mode()[0]
            ):  # Unexpected case
                lower_bound_sparse = nonsparse_data[col].quantile(lower_quantile)
            else:
                lower_bound_sparse = df_outlier_cleaned[col].mode()[0]

            if (  # For upper threshold (right-hand-side)
                nonsparse_data[col].quantile(upper_quantile)
                < df_outlier_cleaned[col].mode()[0]
            ):  # Unexpected case
                upper_bound_sparse = df_outlier_cleaned[col].mode()[0]
            else:
                upper_bound_sparse = nonsparse_data[col].quantile(upper_quantile)

            outlier_boundries_dict[col] = (lower_bound_sparse, upper_bound_sparse)

            # Inform user about the cardinality of Outlier existence:
            number_of_outliers = len(
                df_outlier_cleaned[
                    (df_outlier_cleaned[col] < lower_bound_sparse)
                    | (df_outlier_cleaned[col] > upper_bound_sparse)
                ][col]
            )
            print_if_verbose(
                "Sparse: Outlier number in {} is equal to: ".format(col),
                round(
                    number_of_outliers
                    / (nonsparse_data.shape[0] - nonsparse_data.isnull().sum()),
                    2,
                ),
            )

            # Replace Outliers with Edges --> 1% and 99%:
            if number_of_outliers > 0:

                # Replace 'left-hand-side' outliers with its 1% quantile value
                df_outlier_cleaned.loc[
                    df_outlier_cleaned[col] < lower_bound_sparse, col
                ] = (
                    lower_bound_sparse - epsilon
                )  # --> MAIN DF CHANGED

                # Replace 'right-hand-side' outliers with its 99% quantile value
                df_outlier_cleaned.loc[
                    df_outlier_cleaned[col] > upper_bound_sparse, col
                ] = (
                    upper_bound_sparse + epsilon
                )  # --> MAIN DF CHANGED

        else:  # Find Edges:
            number_of_outliers = len(
                df_outlier_cleaned[
                    (
                        df_outlier_cleaned[col]
                        < df_outlier_cleaned[col].quantile(lower_quantile)
                    )
                    | (
                        df_outlier_cleaned[col]
                        > df_outlier_cleaned[col].quantile(upper_quantile)
                    )
                ][col]
            )
            print_if_verbose(
                "Other: Outlier number in {} is equal to: ".format(col),
                round(
                    number_of_outliers / (df[col].shape[0] - df[col].isnull().sum()), 2
                ),
            )

            # Replace 'Standard' outliers:
            if number_of_outliers > 0:
                # Replace all outliers with its %99 quartile
                lower_bound_sparse = df_outlier_cleaned[col].quantile(lower_quantile)
                df_outlier_cleaned.loc[
                    df_outlier_cleaned[col] < lower_bound_sparse, col
                ] = (lower_bound_sparse - epsilon)

                upper_bound_sparse = df_outlier_cleaned[col].quantile(upper_quantile)
                df_outlier_cleaned.loc[
                    df_outlier_cleaned[col] > upper_bound_sparse, col
                ] = (upper_bound_sparse + epsilon)

            outlier_boundries_dict[col] = (lower_bound_sparse, upper_bound_sparse)

    print_if_verbose("HardEdgeReduction process has been completed!")
    print_if_verbose("--- in %s minutes ---" % ((time.time() - start_time) / 60))

    return df_outlier_cleaned, outlier_boundries_dict


X_train, outlier_boundries_dict = HardEdgeReduction(
    X_train, numerical_columns, sparse_columns
)

print_if_verbose(outlier_boundries_dict)

##  Cleaning Outliers for Test Dataset

epsilon = 0.0001  # for zero divisions

# Define boundaries that we will use for Reduction operation
upper_quantile, lower_quantile = 0.99, 0.01

df_test_outlier_cleaned = X_test.copy()

print_if_verbose(
    "Detected outliers being replaced with edged quantiles/percentiles: 1% and 99%!"
)
print_if_verbose("Total number of rows is: %s\n" % df_test_outlier_cleaned.shape[0])

for col in numerical_columns:
    lower_bound = outlier_boundries_dict[col][0]
    upper_bound = outlier_boundries_dict[col][1]

    # Inform user about the cardinality of Outlier existence:
    number_of_outliers = len(
        df_test_outlier_cleaned[
            (df_test_outlier_cleaned[col] < lower_bound)
            | (df_test_outlier_cleaned[col] > upper_bound)
        ][col]
    )
    print_if_verbose(
        "Outlier number in {} is equal to: ".format(col),
        round(
            number_of_outliers
            / (
                df_test_outlier_cleaned[col].shape[0]
                - df_test_outlier_cleaned[col].isnull().sum()
            ),
            2,
        ),
    )

    # Replace Outliers with Edges --> 1% and 99%:
    if number_of_outliers > 0:
        # Replace 'left-hand-side' outliers with its 1% quantile value
        df_test_outlier_cleaned.loc[df_test_outlier_cleaned[col] < lower_bound, col] = (
            lower_bound - epsilon
        )  # --> MAIN DF CHANGED

        # Replace 'right-hand-side' outliers with its 99% quantile value
        df_test_outlier_cleaned.loc[df_test_outlier_cleaned[col] > upper_bound, col] = (
            upper_bound + epsilon
        )  # --> MAIN DF CHANGED

X_test = df_test_outlier_cleaned

if verbose:
    box_plot(
        Y_train, numerical_columns, X_train
    )  ## Visualization After Cleaning Outlier

print("[INFO] Outlier Detection has been completed.")
###############################################

######## Missing Value Imputation #############
print("[INFO] Missing Value Imputation has begun.")
missing_values_info = df_info["%_Missing_Value"]

Zero_MR_variables_list = list(df_info[missing_values_info == 0].index)
Low_MR_variables_list = list(
    df_info[(missing_values_info > 0) & (missing_values_info <= 0.05)].index
)
Moderate_MR_variables_list = list(
    df_info[(missing_values_info > 0.05) & (missing_values_info <= 0.275)].index
)
High_MR_variables_list = list(
    df_info[(missing_values_info > 0.275) & (missing_values_info <= 0.50)].index
)
Extreme_MR_variables_list = list(
    df_info[(missing_values_info > 0.50) & (missing_values_info <= 0.95)].index
)
Drop_MR_variables_list = list(df_info[missing_values_info > 0.95].index)

print("[INFO] Number of Zero_MR_variables_list:", len(Zero_MR_variables_list))
print("[INFO] Number of Low_MR_variables_list:", len(Low_MR_variables_list))
print("[INFO] Number of Moderate_MR_variables_list:", len(Moderate_MR_variables_list))
print("[INFO] Number of High_MR_variables_list:", len(High_MR_variables_list))
print("[INFO] Number of Extreme_MR_variables_list:", len(Extreme_MR_variables_list))


assert len(Zero_MR_variables_list) + len(Low_MR_variables_list) + len(
    Moderate_MR_variables_list
) + len(High_MR_variables_list) + len(Extreme_MR_variables_list) == len(df_info)

# Simple Imputer for Low Missing Values


def SimpleImputer(df, data_info, variable_list):
    for col in variable_list:

        if col in numerical_columns:

            print_if_verbose(
                "Total null values: {}".format(df[[str(col)]].isnull().sum())
            )

            average = float(df[col].mean())
            std = float(df[col].std())
            count_nan = int(df[col].isnull().sum())
            rand = np.random.normal(loc=average, scale=std, size=count_nan)
            slice_col = pd.Series(df[col].copy())
            slice_col[pd.isnull(slice_col)] = rand
            df[col] = slice_col

            print_if_verbose("Numerical variable {} have been imputed.".format(col))

        else:

            print_if_verbose(
                "Total null values: {}".format(df[[str(col)]].isnull().sum())
            )
            df.loc[df.loc[:, col].isnull(), col] = np.random.choice(
                sorted(list(df.loc[:, col].dropna().unique())),
                size=int(df.loc[df.loc[:, col].isnull(), col].shape[0]),
                p=[
                    pd.Series(
                        df.groupby(col).size() / df.loc[:, col].dropna().shape[0]
                    ).iloc[i]
                    for i in np.arange(0, len(df.loc[:, col].dropna().unique()))
                ],
            )

            print_if_verbose("Categorical variable {} have been imputed.".format(col))


print_if_verbose(Low_MR_variables_list)

SimpleImputer(X_train, df_info, Low_MR_variables_list)
SimpleImputer(X_test, df_info, Low_MR_variables_list)

print_if_verbose(MissingUniqueStatistics(X_train.loc[:, Low_MR_variables_list]))
print_if_verbose(MissingUniqueStatistics(X_test.loc[:, Low_MR_variables_list]))

print("[INFO] Missing Value Imputation has been completed.")
###############################################

Sparse Columns: []
[INFO] Outlier Detection has been completed.
[INFO] Missing Value Imputation has begun.
[INFO] Number of Zero_MR_variables_list: 16
[INFO] Number of Low_MR_variables_list: 3
[INFO] Number of Moderate_MR_variables_list: 7
[INFO] Number of High_MR_variables_list: 0
[INFO] Number of Extreme_MR_variables_list: 0
[INFO] Missing Value Imputation has been completed.


<IPython.core.display.Javascript object>

In [13]:
######## Categorical Value Encodings #########
### MEAN ENCODING

class KFoldTargetEncoderTrain(base.BaseEstimator,
                               base.TransformerMixin):
    def __init__(self,colnames,targetName,
                  n_fold=5, verbosity=True,
                  discardOriginal_col=False):
        self.colnames = colnames
        self.targetName = targetName
        self.n_fold = n_fold
        self.verbosity = verbosity
        self.discardOriginal_col = discardOriginal_col
    
    def fit(self, X, y=None):
        return self
    
    def transform(self,X):
        assert(type(self.targetName) == str)
        assert(type(self.colnames) == str)
        assert(self.colnames in X.columns)
        assert(self.targetName in X.columns)
        
        mean_of_target = X[self.targetName].mean()
        kf = KFold(n_splits = self.n_fold,
                   shuffle = True, random_state=42)
        col_mean_name = self.colnames + '_' + 'Kfold_Target_Enc'
        X[col_mean_name] = np.nan
        for tr_ind, val_ind in kf.split(X):
            X_tr, X_val = X.iloc[tr_ind], X.iloc[val_ind]
            X.loc[X.index[val_ind], col_mean_name] = \
            X_val[self.colnames].map(X_tr.groupby(self.colnames)
                                     [self.targetName].mean())
            X[col_mean_name].fillna(mean_of_target, inplace = True)
        if self.verbosity:
            encoded_feature = X[col_mean_name].values
            print_if_verbose('Correlation between the new feature, {} and, {} is {}.'\
                  .format(col_mean_name,self.targetName,
                          np.corrcoef(X[self.targetName].values,
                                      encoded_feature)[0][1]))
        if self.discardOriginal_col:
            X = X.drop(self.targetName, axis=1)
        return X
def StringConverterTrain(df,target_name,variable_list):
    for col in variable_list:
        targetc = KFoldTargetEncoderTrain(col,target_name,n_fold=4)
        new_train = targetc.fit_transform(df)
    return new_train


nominal_variable = list(df_info[df_info["Variable_Type"] == "Nominal"].index)
nominal_lst = [
    item
    for item in Moderate_MR_variables_list
    + High_MR_variables_list
    + Extreme_MR_variables_list
    if item in nominal_variable
]
if "OFFER_STATUS" in nominal_lst:
    nominal_lst.remove("OFFER_STATUS")

nominal_lst

df_trial = pd.concat([X_train, Y_train], axis=1).copy()
df_output_train = StringConverterTrain(
    df=df_trial, target_name="OFFER_STATUS", variable_list=nominal_lst
)

for item in nominal_lst:
    print_if_verbose(df_output_train.loc[:, [item + "_Kfold_Target_Enc"]].isnull().sum())

for item in nominal_lst:
    X_train[item] = df_output_train[item + "_Kfold_Target_Enc"]

### Mean Encoding for nominal variables(non missing value)

df_encoding = pd.concat([X_train, Y_train], axis=1).copy()
categorical_non_missing_columns = type_separator(df_encoding[get_non_null_columns(df_encoding)])["categorical"]

if "OFFER_STATUS" in categorical_non_missing_columns:
    categorical_non_missing_columns.remove("OFFER_STATUS")
print_if_verbose(categorical_non_missing_columns)

df_encoding_train = StringConverterTrain(
    df=df_encoding,
    target_name="OFFER_STATUS",
    variable_list=categorical_non_missing_columns,
)

for item in categorical_non_missing_columns:
    X_train[item] = df_encoding_train[item + "_Kfold_Target_Enc"]

for item in categorical_non_missing_columns:
    print_if_verbose(df_encoding_train.loc[:, [item + "_Kfold_Target_Enc"]].isnull().sum())

MissingUniqueStatistics(X_train)

# **String Converter for Test Dataset**

df_output_test = X_test.copy()
mean_of_target = df_output_train["OFFER_STATUS"].copy().mean()
target_mean_list = nominal_lst
for col in target_mean_list:
    df_output_test[col] = df_output_test[col].map(
        df_output_train.groupby(col)[col + "_Kfold_Target_Enc"].mean()
    )
    df_output_test[col].fillna(mean_of_target, inplace=True)

for item in nominal_lst:
    print_if_verbose(df_output_test.loc[:, [item]].isnull().sum())

X_test[nominal_lst] = df_output_test[nominal_lst]

X_test_encoder = X_test.copy()
mean_of_target = df_encoding_train["OFFER_STATUS"].copy().mean()
target_mean_list = categorical_non_missing_columns
for col in target_mean_list:
    X_test_encoder[col + "_Kfold_Target_Enc"] = X_test_encoder[col].map(
        df_encoding_train.groupby(col)[col + "_Kfold_Target_Enc"].mean()
    )
    X_test_encoder[col + "_Kfold_Target_Enc"].fillna(mean_of_target, inplace=True)

for item in categorical_non_missing_columns:
    X_test[item] = X_test_encoder[item + "_Kfold_Target_Enc"]

MissingUniqueStatistics(X_test[categorical_non_missing_columns])

## Modal Based Imputation

def MBI(df, columns, train_or_test, lst_numerical):

    data_binary_encoded = df.copy()
    le = LabelEncoder()

    if columns:
        for col in columns:
            if train_or_test == "test":
                le.fit(X_train[col].copy().astype(str))
                data_binary_encoded[col] = le.transform(df[col].copy().astype(str))
            else:
                data_binary_encoded[col] = le.fit_transform(df[col].copy().astype(str))

    data_scaled = data_binary_encoded.copy()

    for col in numerical_columns:

        scaler = StandardScaler()

        if train_or_test == "test":

            scaler.fit(np.array(X_train.loc[:, col]).reshape(-1, 1))
            data_scaled.loc[:, col] = scaler.transform(
                np.array(data_scaled.loc[:, col]).reshape(-1, 1)
            )

        else:
            data_scaled.loc[:, col] = scaler.fit_transform(
                np.array(data_scaled.loc[:, col]).reshape(-1, 1)
            )

    for col in lst_numerical:

        target_dropped_fullcases = (
            data_scaled.drop(col, axis=1)
            .loc[
                :,
                list(
                    set(Zero_MR_variables_list + Low_MR_variables_list)
                    - set(["CLASS", "KEY", "CLNHGVS"])
                ),
            ]
            .copy()
        )

        target = data_scaled.loc[:, col]
        null_mask = target.isna()
        print_if_verbose(col)

        if col in numerical_columns:

            mlp = MLPRegressor(
                hidden_layer_sizes=(
                    100,
                    10,
                ),
                activation="tanh",
                solver="adam",
                learning_rate="adaptive",
                max_iter=1000,
                learning_rate_init=0.01,
                alpha=0.01,
                early_stopping=False,
            )
        else:
            mlp = MLPClassifier(
                hidden_layer_sizes=(
                    100,
                    10,
                ),
                activation="tanh",
                solver="adam",
                learning_rate="adaptive",
                max_iter=1000,
                learning_rate_init=0.01,
                alpha=0.01,
                early_stopping=False,
            )

        mlp.fit(target_dropped_fullcases[~null_mask], target[~null_mask])
        data_scaled.loc[null_mask, col] = mlp.predict(
            target_dropped_fullcases[null_mask]
        )

    print_if_verbose(data_scaled.loc[:, lst_numerical].isnull().sum())
    return data_scaled

moderate_numerical_variables = [
    item for item in Moderate_MR_variables_list if item in numerical_columns
]


<IPython.core.display.Javascript object>

In [14]:
moderate_numerical_variables

assert (
    "categorical" not in type_separator(X_train)
    or not type_separator(X_train)["categorical"]
), print_if_verbose("All categorical variables should have encoded, check encodings!")
assert (
    "categorical" not in type_separator(X_test)
    or not type_separator(X_test)["categorical"]
), print_if_verbose("All categorical variables should have encoded, check encodings!")

X_train_scaled = MBI(X_train, None, "train", moderate_numerical_variables)

X_train_scaled

X_test_scaled = MBI(X_test, None, "test", moderate_numerical_variables)

X_test_scaled

MissingUniqueStatistics(X_train_scaled)

assert get_null_columns(X_train_scaled) == [], print_if_verbose(
    "There are some missing values left!"
)
assert get_null_columns(X_test_scaled) == [], print_if_verbose(
    "There are some missing values left!"
)


###############################################

<IPython.core.display.Javascript object>

In [ ]:
rnd_clf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42)
rnd_clf.fit(X_train_scaled, Y_train)

features = X_train_scaled.columns
importances = rnd_clf.feature_importances_
indices = np.argsort(importances)

plt.figure(figsize=(20, 10))
feat_importances = pd.Series(importances, index=features)
feat_importances.nlargest(len(indices)).plot(kind="bar", color="#79CCB3")

In [40]:
resultt = apply_h2o(
    data=[X_train_scaled, X_test_scaled, Y_train, Y_test],
    max_runtime_secs=5 * 60 * 60,
    training_with_all=True,
)

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,58 mins 04 secs
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.2
H2O_cluster_version_age:,3 days
H2O_cluster_name:,H2O_from_python_iceking_xb133p
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,31.74 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,58 mins 07 secs
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.2
H2O_cluster_version_age:,3 days
H2O_cluster_name:,H2O_from_python_iceking_xb133p
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,31.74 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


[INFO] Model will be saved here:"./h2o_models_with_data/results_ts_2022-01-29_18:21:15/model_withALL"
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
[INFO] Timestamp: 2022-01-29_18:21:15
[INFO] AML Leaderboard 

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_2_AutoML_17_20220129_182118,0.809839,0.385342,0.935896,0.380876,0.345134,0.119118
StackedEnsemble_AllModels_3_AutoML_17_20220129_182118,0.809716,0.384934,0.936061,0.372267,0.344993,0.11902
StackedEnsemble_AllModels_7_AutoML_17_20220129_182118,0.809687,0.384936,0.936212,0.376767,0.345055,0.119063
StackedEnsemble_AllModels_4_AutoML_17_20220129_182118,0.809365,0.385508,0.935867,0.371232,0.345202,0.119164
StackedEnsemble_BestOfFamily_3_AutoML_17_20220129_182118,0.809289,0.386617,0.935803,0.381496,0.345501,0.119371
StackedEnsemble_AllModels_1_AutoML_17_20220129_182118,0.808943,0.385722,0.935752,0.367009,0.345463,0.119345
StackedEnsemble_BestOfFamily_4_AutoML_17_20220129_182118,0.808859,0.386183,0.935928,0.370963,0.345305,0.119236
StackedEnsemble_BestOfFamily_7_AutoML_17_20220129_182118,0.808694,0.386663,0.935268,0.381686,0.345341,0.11926
StackedEnsemble_AllModels_6_AutoML_17_20220129_182118,0.807609,0.385613,0.935052,0.365562,0.3459,0.119647
StackedEnsemble_BestOfFamily_2_AutoML_17_20220129_182118,0.807503,0.387193,0.935108,0.379517,0.345892,0.119641



[INFO] RESULTS:
 > Train data saved to:"./h2o_models_with_data/results_ts_2022-01-29_18:21:15/train.csv".
 > Test data saved to :"./h2o_models_with_data/results_ts_2022-01-29_18:21:15/test.csv".
 > H20 Model saved to :"./h2o_models_with_data/results_ts_2022-01-29_18:21:15/model_withALL".


<IPython.core.display.Javascript object>

In [ ]:
resultt

In [ ]:
use_auto_sklearn=False
if use_auto_sklearn:
    x_train_data = X_train_scaled.copy()
    x_test_data = X_test_scaled.copy()

    model2 = AutoSklearnClassifier(
        time_left_for_this_task=2 * 60,
        # time_left_for_this_task=60,
        seed=42,
        per_run_time_limit=30,
        n_jobs=-1,
        metric=autosklearn.metrics.balanced_accuracy,
        memory_limit=10 * 3072,
        # resampling_strategy="cv",
        # resampling_strategy_arguments={"cv": {"folds": 5}, "shuffle": True},
        # resampling_strategy_arguments={"cv": {"folds": 10}, "shuffle": True},
        # resampling_strategy_arguments={"train_size": 0.67, "shuffle": True},
    )

    start = time.time()
    model2.fit(x_train_data, Y_train)
    end = time.time()
    y_hat = model2.predict(x_test_data)
    # print(model2.sprint_statistics())
    # print(model2.best_val_acc)
    best_val_score_here = float(model2.sprint_statistics().split("\n")[3][25:])
    print(model2.get_models_with_weights())
    # model2.leaderboard()


    # Evaluate

    print("Elapsed Time:", end - start)
    # print("Dropped until:", col)
    print("Best Val Score:", best_val_score_here)
    val_acc = balanced_accuracy_score(y_true=Y_test, y_pred=y_hat)
    print("Final BAC Score: %.3f" % val_acc)

    # Pickle
    if not (("best_val_acc" in vars() or "best_val_acc" in globals())):
        best_val_acc = val_acc
    file_name = f"model_pickles/model_{int(val_acc*10**3)}" + str(
        datetime.datetime.now()
    ).replace(" ", "_")
    if val_acc >= best_val_acc:
        with open(file_name, "wb") as f:
            pickle.dump(model2, f)
    else:
        with open(file_name, "wb") as f:
            pickle.dump(model2, f)

In [ ]:
## gggggg

In [ ]:
from sklearn.ensemble import RandomForestClassifier

X_for_boruta = pd.concat([X_train_scaled, X_test_scaled]).copy()
Y_for_boruta = pd.concat([Y_train, Y_test]).copy()
# define random forest classifier
forest = RandomForestClassifier(n_jobs=-1, class_weight="balanced")
forest.fit(X_for_boruta, Y_for_boruta)


from boruta import BorutaPy

# define Boruta feature selection method
feat_selector = BorutaPy(
    forest,
    n_estimators="auto",
    verbose=2,
    random_state=42,
)


feat_selector.fit(
    X_for_boruta.to_numpy(), Y_for_boruta.to_numpy()
)  # find all relevant features
print(feat_selector.support_)  # check selected features
print(feat_selector.ranking_)  # check ranking of features
# check selected features
a = list(
    zip(
        list(feat_selector.ranking_),
        list(feat_selector.support_),
        list(X_for_boruta.columns),
    )
)

[", # ".join(map(str, [f'{x[2]}", # ', x[0], x[1]])) for x in sorted(a)]

In [ ]:
Best Val Score: 0.70742
Final BAC Score: 0.713

### real predictt

In [42]:
def real_predict_h2o(models_dir_path):
    from h2o.automl import H2OAutoML
    import h2o

    h2o.init(max_mem_size="32G")

    my_model = h2o.load_model(models_dir_path)
    df = pd.read_csv("interim_data/df_completed_1_2_3_with_mv_new.csv")

    ppppp = df_for_unlabeled_set.drop(["OFFER_STATUS"], axis=1)
    ppppp = X_test_scaled
    x_realll = h2o.H2OFrame(ppppp)

    ggl = my_model.predict(x_realll)
    gg = ggl[0].as_data_frame().values.flatten()

    real_test = ppppp
    real_test["prediction"] = gg
    test_set_id_col = df[np.isnan(df["OFFER_STATUS"])]["TEST_SET_ID"]
    real_test = pd.concat([real_test["prediction"], test_set_id_col], axis=1)
    real_test = real_test.rename(columns={"TEST_SET_ID": "id"})

    real_test["prediction"] = real_test["prediction"].astype(int)
    real_test["id"] = real_test["id"].astype(int)
    print(ggl)

    return real_test


lllresult = real_predict_h2o(
    # Submission 5
    # models_dir_path="h2o_models_with_data/results_ts_2022-01-29_13:09:41/model_bac_0.734/GBM_1_AutoML_31_20220129_130944"
    # Submission 7
    models_dir_path="./h2o_models_with_data/results_ts_2022-01-29_18:21:15/model_withALL/StackedEnsemble_AllModels_2_AutoML_17_20220129_182118"
)

print(lllresult.prediction.value_counts())

lllresult.to_csv("predictions_versed_chimpanzee_7.csv", header=True, index=False)

lllresult

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,2 hours 30 mins
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.2
H2O_cluster_version_age:,3 days
H2O_cluster_name:,H2O_from_python_iceking_xb133p
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,30.89 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


predict,p0,p1
1,0.18431,0.81569
1,0.320166,0.679834
1,0.285917,0.714083
1,0.258053,0.741947
1,0.410835,0.589165
1,0.300396,0.699604
1,0.320632,0.679368
1,0.382059,0.617941
1,0.266544,0.733456
1,0.170701,0.829299



1    2517
0    59  
Name: prediction, dtype: int64


,prediction,id
5,1,6
8,1,9
13,1,14
34,1,35
35,1,36
...,...,...
26018,1,26019
26032,1,26033
26049,1,26050
26057,1,26058


<IPython.core.display.Javascript object>

In [ ]:
lllresult.as_data_frame().head(30)

In [ ]:
pd.Series(ggggggggggggg).value_counts()

In [ ]:
    df_tmp = pd.read_csv("interim_data/df_completed_1_2_3_with_mv_new.csv")
    test_set_id_col = df_tmp[np.isnan(df_tmp["OFFER_STATUS"])]["TEST_SET_ID"]

    ## Predict BREAK - start
    real_test = x_test_data
    real_test["prediction"] = model2.predict(real_test)
    real_test = pd.concat([real_test["prediction"], test_set_id_col], axis=1)
    real_test = real_test.rename(columns={"TEST_SET_ID": "id"})
    real_test["prediction"] = real_test["prediction"].astype(int)
    real_test["id"] = real_test["id"].astype(int)

    ## Predict BREAK - stop

    #real_test.to_csv("predictions_versed_chimpanzee_5.csv", header=True, index=False)
    print(real_test[["prediction"]].value_counts())